# **Link e funzioni di base**

In [ ]:
#Librerie utili
import pandas as pd
import numpy as np
from copy import deepcopy

import string
import re

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

#!pip install py_stringsimjoin
#import py_stringsimjoin as ssj
# a commento perchè non funziona
!pip install git+https://github.com/anhaidgroup/py_stringsimjoin
import py_stringmatching as sm
import py_stringsimjoin as ssj
import warnings
import requests
import networkx as nx
warnings.filterwarnings('ignore')

  Cloning https://github.com/anhaidgroup/py_stringsimjoin to /tmp/pip-req-build-lvw6dppm
  Running command git clone --filter=blob:none --quiet https://github.com/anhaidgroup/py_stringsimjoin /tmp/pip-req-build-lvw6dppm
  Resolved https://github.com/anhaidgroup/py_stringsimjoin to commit fb5b9745829a873481687eb2e0e1e2223a7e733a
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for py-stringsimjoin: filename=py_stringsimjoin-0.3.5-cp310-cp310-linux_x86_64.whl size=5282911 sha256=fa58b27aa78398ca30d0d8b0b38ce428dd65ba40ec6e768491334a552d8a7204
  Stored in directory: /tmp/pip-ephem-wheel-cache-c89744x1/wheels/f6/da/a6/cc24da6a2843fa3eec0f2801905be0736899c8caab75c050ef
  Created wheel for py_stringmatching: filename=py_stringmatching-0.4.5-cp310-cp310-linux_x86_64.whl size=2834017 sha256=5064bdef3f90f87e73f456229c5b0a488df60a607c8f177cd736df4bfddb4d7d
  Stored 

In [ ]:
################
#funzioni utili#
################
#funzione di conversione da  tabella a matrice
def sim_table(TableA:pd.DataFrame, TableB:pd.DataFrame):
    A = pd.DataFrame({"A": TableA.columns})
    B = pd.DataFrame({"B": TableB.columns})
    S = A.merge(B, how='cross')
    return S


#funzione che crea una sim matrix vuota
def sim_matrix(TableA:pd.DataFrame, TableB:pd.DataFrame):
    S=pd.DataFrame(columns=TableA.columns.tolist(),index=TableB.columns.tolist())
    return S

#creiamo una sim table con valori randomici
def random_sim_table(TableA:pd.DataFrame, TableB:pd.DataFrame):
    S = sim_table(TableA, TableB)
    S["sim"] = np.random.rand(len(S))
    return S

#convertiamo una sim matrix a sim table
def to_sim_table(SimMatrix:pd.DataFrame):
    return SimMatrix.stack().reset_index(name="sim")

#versione con rename e reorder
def to_sim_table_rename(SimMatrix:pd.DataFrame):
    #parametro name per il valore
    #rename per cambiare il nome delle colonne degli indici
    #nelle quadre specifichiamo l'ordine voluto
    return SimMatrix.stack().reset_index(name='sim').rename(columns={'level_0':'B','level_1':'A'})[['A','B','sim']]

#convertiamo una sim table a sim matrix
def to_sim_matrix(SimTable:pd.DataFrame):
    return SimTable.pivot(index="A", columns="B", values="sim") \
              .rename_axis(None, axis=1).rename_axis(None, axis=0)

def prodotto_cartesiano(DA:pd.DataFrame,DB:pd.DataFrame):
    #prendiamo 2 dataframe, aggiungiamo una colonna di 1, con label key, facciamo il merge
    #e droppiamo 'key'
    return DA.assign(key=1).merge(DB.assign(key=1), on="key").drop(columns='key')

#versione che droppa dupplicati
def prodotto_cartesiano_no_dupes(DA:pd.DataFrame,DB:pd.DataFrame):
    return  DA.drop_duplicates().merge(DB.drop_duplicates(), how='cross')

#join esatto
def join_esatto(DA:pd.DataFrame,DB:pd.DataFrame):
    PCC=prodotto_cartesiano_no_dupes(DA,DB)
    return PCC[PCC.AX==PCC.AY].drop_duplicates()
#elementi presenti in una sola tabella dato un join
def solo_in(PCC:pd.DataFrame,Column):
    return PCC.loc[~PCC[Column].isin(INTERSEZIONE[Column])][[Column]].drop_duplicates()

In [ ]:
#########################
#Funzioni di valutazione#
#########################

#Confrontiamo i match con il gold standard
def Valuta(Gold:pd.DataFrame, Match:pd.DataFrame):
    Match = Match[['A','B']]
    FOJ = Gold.merge(Match, how='outer', indicator=True)

    TP = FOJ[FOJ['_merge']=='both']
    FP = FOJ[FOJ['_merge']=='right_only']
    FN = FOJ[FOJ['_merge']=='left_only']

    if len(TP) == 0:
        return pd.DataFrame({
                'MT':[len(Match)],
                'TP':[len(TP)],
                'FP':[len(FP)],
                'FN':[len(FN)],
                'P':[round(0,4)],
                'R':[round(0,4)],
                'F':[round(0,4)]
            })
    else:
        P = len(TP)/(len(TP)+len(FP))
        R = len(TP)/(len(TP)+len(FN))
        F = 2 * P * R / ( P + R )
        return pd.DataFrame({
                'MT':[len(Match)],
                'TP':[len(TP)],
                'FP':[len(FP)],
                'FN':[len(FN)],
                'P':[round(P,4)],
                'R':[round(R,4)],
                'F':[round(F,4)]
            })

#chiediamo le metriche di valutazione come FP/TN/TP...
def Vedi_Valuta(Gold:pd.DataFrame, Match:pd.DataFrame, metrics:str):
    Match=Match[['A','B']]
    FOJ=pd.merge(Gold, Match, how='outer', indicator=True)

    TP=FOJ[FOJ['_merge']=='both']
    FP=FOJ[FOJ['_merge']=='right_only']
    FN=FOJ[FOJ['_merge']=='left_only']

    if metrics == 'FP' :
        return FP
    if metrics == 'TP' :
        return TP
    if metrics == 'FN' :
        return FN

In [ ]:
##################################
#funzioni di string preprocessing#
##################################
def preprocess_s(s):
  # un minimo di Pre-Processing of Labels
      if type(s) is str:
        s = s.lower()
        s = s.replace(',', ' ')
        s = s.replace('&', 'and')
        s = s.replace('_', '')
        s = s.replace('?', '')
        s = s.replace('movie', '') # aggiunta nell'esempio movie
      return s.strip()

def string_preprocess(s:str, char:str=string.punctuation, word:list=[]):
  # una migliore e più completa funzione di Pre-Processing è nella tesina di Mirco Botti
    if type(s) is str:
        s = s.lower()
        for c in char:
            s = s.replace(c, " ")
        for w in word:
            s = s.replace(w, " ")
    else:
        s = str(s)
    s = re.sub(" +", " ", s)
    return s.strip()

# **Funzioni di similarità**

## **Funzioni di similarità label based**

In [ ]:
def levenshtein_sim(row:pd.Series):
    lev = sm.Levenshtein()
    return lev.get_sim_score(
            string_preprocess(row["A"]),
            string_preprocess(row["B"])
        )

def levenshtein_label_based_similarity(TableA:pd.DataFrame, TableB:pd.DataFrame):
    C = sim_table(TableA, TableB)
    C["sim"] = C.apply(levenshtein_sim, axis=1)
    return C.sort_values("sim", ascending=False)

# Jaro
def jaro_sim(row:pd.Series):
    jaro = sm.Jaro()
    return jaro.get_sim_score(
            string_preprocess(row["A"]),
            string_preprocess(row["B"])
        )

def jaro_label_based_similarity(TableA:pd.DataFrame,TableB:pd.DataFrame):
    C = sim_table(TableA, TableB)
    C["sim"] = C.apply(jaro_sim, axis=1)
    return C


# Jaccard
def jaccard_sim(row:pd.Series):
    jac=sm.Jaccard()
    tok = sm.WhitespaceTokenizer(return_set=True)
    return jac.get_sim_score(
            tok.tokenize(string_preprocess(row["A"])),
            tok.tokenize(string_preprocess(row["B"]))
    )

def jaccard_label_based_similarity(TableA:pd.DataFrame, TableB:pd.DataFrame):
    C = sim_table(TableA, TableB)
    C["sim"] = C.apply(jaccard_sim, axis=1)
    return C.sort_values("sim", ascending=False)

# OverlapCoefficient
def OC_sim(row:pd.Series):
    oc = sm.OverlapCoefficient()
    tok = sm.WhitespaceTokenizer(return_set=True)
    return oc.get_sim_score(
            tok.tokenize(string_preprocess(row["A"])),
            tok.tokenize(string_preprocess(row["B"]))
    )

def OC_label_based_similarity(TableA:pd.DataFrame, TableB:pd.DataFrame):
    C = sim_table(TableA, TableB)
    C["sim"] = C.apply(OC_sim, axis=1)
    return C.sort_values("sim", ascending=False)

# JaroWinkler
def JaroWinkler_sim(row:pd.Series):
    jw = sm.JaroWinkler()
    return jw.get_sim_score(
            string_preprocess(row["A"]),
            string_preprocess(row["B"])
        )

def JaroWinkler_label_based_similarity(TableA:pd.DataFrame, TableB:pd.DataFrame):
    C = sim_table(TableA, TableB)
    C["sim"] = C.apply(JaroWinkler_sim, axis=1)
    return C.sort_values("sim", ascending=False)

# MongeElkan
#Secondary similarity function. This is expected to be a sequence-based similarity measure
#(defaults to Jaro-Winkler similarity measure).
def MongeElkan_sim(row:pd.Series):
    me = sm.MongeElkan()
    tok = sm.WhitespaceTokenizer(return_set=True)
    return me.get_raw_score(
            tok.tokenize(string_preprocess(row["A"])),
            tok.tokenize(string_preprocess(row["B"]))
    )


def MongeElkan_label_based_similarity(TableA:pd.DataFrame, TableB:pd.DataFrame):
    C = sim_table(TableA, TableB)
    C["sim"] = C.apply(MongeElkan_sim, axis=1)
    return C.sort_values("sim", ascending=False)

## **Funzioni di similarità Value Overlap**

In [ ]:
#funzioni che controllano se i valori sono sovrapposti
def jaccard_sim_value(row:pd.Series, TableA:pd.DataFrame,TableB:pd.DataFrame):
    j = sm.Jaccard()
    return j.get_raw_score(
            TableA[row["A"]].apply(string_preprocess).tolist(),
            TableB[row["B"]].apply(string_preprocess).tolist()
        )

def jaccard_value_overlap_sim(TableA:pd.DataFrame, TableB:pd.DataFrame):
    C = sim_table(TableA, TableB)
    C["sim"] = C.apply(jaccard_sim_value, args=(TableA, TableB), axis=1)
    return C.sort_values("sim", ascending=False)

def generalized_sim_value(row:pd.Series, TableA:pd.DataFrame, TableB:pd.DataFrame, threshold:float):
    j = sm.GeneralizedJaccard(
            sim_func=sm.Levenshtein().get_sim_score,
            threshold=threshold
        )
    return j.get_raw_score(
            TableA[row["A"]].apply(string_preprocess).tolist(),
            TableB[row["B"]].apply(string_preprocess).tolist()
        )

def generalized_value_overlap_sim(TableA:pd.DataFrame, TableB:pd.DataFrame, threshold:float):
    C = sim_table(TableA, TableB)
    C["sim"] = C.apply(generalized_sim_value, args=(TableA, TableB, threshold), axis=1)
    return C.sort_values("sim", ascending=False)

In [ ]:
#extended Jaccard, non fa il confronto totale
def funzione_similarita_internaLEV(row:pd.Series): # Levenshtein
    lev = sm.Levenshtein()
    return lev.get_sim_score(
            string_preprocess(row["AX"]),
            string_preprocess(row["AY"])
        )

def extended_value_overlap_sim_LEV(row:pd.Series, TableA:pd.DataFrame,TableB:pd.DataFrame, threshold:float):
    TX = TableA[[row["A"]]].applymap(string_preprocess).drop_duplicates()
    TY = TableB[[row["B"]]].applymap(string_preprocess).drop_duplicates()
    TX.columns=['AX']
    TY.columns=['AY']
    PCC = TX.drop_duplicates().merge(TY.drop_duplicates(), how='cross')
    PCC["SimJac"] = PCC.apply(funzione_similarita_internaLEV, axis=1)
    INTERSEZIONE =  PCC[PCC.SimJac>=threshold]
    SoloInAX=PCC.loc[~PCC['AX'].isin(INTERSEZIONE['AX'])][['AX']].drop_duplicates()
    SoloInAY=PCC.loc[~PCC['AY'].isin(INTERSEZIONE['AY'])][['AY']].drop_duplicates()
    return len(INTERSEZIONE)/(len(SoloInAX)+len(SoloInAY)+len(INTERSEZIONE))

def value_overlap_extended_jaccard_LEV(TableA:pd.DataFrame, TableB:pd.DataFrame, threshold:float):
    C = sim_table(TableA, TableB)
    C["sim"] = C.apply(extended_value_overlap_sim_LEV, args=(TableA, TableB, threshold), axis=1)
    return C.sort_values("sim",ascending=False)

def funzione_similarita_internaJaccard(row:pd.Series): # Jaccard
    jac=sm.Jaccard()
    tok = sm.WhitespaceTokenizer(return_set=True)
    return jac.get_sim_score(
            tok.tokenize(string_preprocess(row["AX"])),
            tok.tokenize(string_preprocess(row["AY"])))

def extended_value_overlap_sim_JAC(row:pd.Series, TableA:pd.DataFrame,TableB:pd.DataFrame, threshold:float):
    TX = TableA[[row["A"]]].applymap(string_preprocess).drop_duplicates()
    TY = TableB[[row["B"]]].applymap(string_preprocess).drop_duplicates()
    TX.columns=['AX']
    TY.columns=['AY']
    PCC = TX.drop_duplicates().merge(TY.drop_duplicates(), how='cross')
    PCC["SimJac"] = PCC.apply(funzione_similarita_internaJaccard, axis=1)
    INTERSEZIONE =  PCC[PCC.SimJac>=threshold]
    SoloInAX=PCC.loc[~PCC['AX'].isin(INTERSEZIONE['AX'])][['AX']].drop_duplicates()
    SoloInAY=PCC.loc[~PCC['AY'].isin(INTERSEZIONE['AY'])][['AY']].drop_duplicates()
    return len(INTERSEZIONE)/(len(SoloInAX)+len(SoloInAY)+len(INTERSEZIONE))

def value_overlap_extended_jaccard_JAC(TableA:pd.DataFrame, TableB:pd.DataFrame, threshold:float):
    C = sim_table(TableA, TableB)
    C["sim"] = C.apply(extended_value_overlap_sim_JAC, args=(TableA, TableB, threshold), axis=1)
    return C.sort_values("sim",ascending=False)

In [ ]:
def sim__join(row:pd.Series, TableA:pd.DataFrame,TableB:pd.DataFrame, threshold:float):
    TX = TableA[[row["A"]]].applymap(string_preprocess).drop_duplicates()
    TY = TableB[[row["B"]]].applymap(string_preprocess).drop_duplicates()
    TX.columns=['AX']
    TY.columns=['AY']

    INTERSEZIONE  = ssj.jaccard_join(     TX, TY, # tabelle su cui effettuare il sim join
                                'AX', 'AY', # chiavi delle tabelle
                                'AX', 'AY', # attributi di join
                                  sm.WhitespaceTokenizer(return_set=True),
                                  threshold=threshold,
                                  show_progress=False,
                                  l_out_attrs=['AX'],  r_out_attrs=['AY']
                           )
    SoloInAX=TX.loc[~TX['AX'].isin(INTERSEZIONE['l_AX'])][['AX']].drop_duplicates()
    SoloInAY=TY.loc[~TY['AY'].isin(INTERSEZIONE['r_AY'])][['AY']].drop_duplicates()
    return len(INTERSEZIONE)/(len(SoloInAX)+len(SoloInAY)+len(INTERSEZIONE))


def value_overlap_simjoin_jaccard(TableA:pd.DataFrame, TableB:pd.DataFrame, threshold:float):
    C = sim_table(TableA, TableB)
    C["sim"] = C.apply(sim__join, args=(TableA, TableB, threshold), axis=1)
    return C.sort_values("sim",ascending=False)

# **Combined aproach**

In [ ]:
#approcci combinati per le tabelle di similarità

#LocalMatchingTable = pd.concat([LocalMatchingTable, LocalMatchingTableXY], ignore_index=True, sort=True)
#passiamo una lista di tabelle e ritorno il massimo
def max_sim_table(SimTableList:list):
    ST = pd.DataFrame(columns=["A","B","sim"])
    for x in SimTableList:
        #ST = ST.append(x, ignore_index=True)
        ST = pd.concat([ST, x], ignore_index=True)
    return ST.groupby(["A","B"])["sim"].max().reset_index()

def min_sim_table(SimTableList:list):
    ST = pd.DataFrame(columns=["A","B","sim"])
    for x in SimTableList:
        #ST = ST.append(x, ignore_index=True)
        ST = pd.concat([ST, x], ignore_index=True)
    return ST.groupby(["A","B"])["sim"].min().reset_index()

def avg_sim_table(SimTableList:list):
    ST = pd.DataFrame(columns=["A","B",'sim'])
    for x in SimTableList:
        #ST = ST.append(x, ignore_index=True)
        ST = pd.concat([ST, x], ignore_index=True)
    return ST.groupby(["A","B"])["sim"].mean().reset_index()

In [ ]:
def  Weighted_sum(dataframes, weights):
    if len(dataframes) < 2 or len(dataframes) != len(weights):
        raise ValueError("È necessario fornire almeno due DataFrame e i relativi pesi.")

    result_df = dataframes[0].copy()

    result_df["sim"] = result_df["sim"] * weights[0]

    for i in range(1, len(dataframes)):
        df = dataframes[i]
        weight = weights[i]

        result_df = pd.merge(result_df, df, on=["A", "B"], how="outer")

        result_df["sim"] = result_df.apply(lambda row: row["sim_x"] + row["sim_y"] * weight if not pd.isnull(row["sim_x"]) and not pd.isnull(row["sim_y"]) else row["sim_x"] if not pd.isnull(row["sim_x"]) else row["sim_y"] * weight, axis=1)

        result_df = result_df.drop(columns=["sim_x", "sim_y"])

    return result_df

# **Funzioni generali**

In [ ]:
def to_GMT(MT:pd.DataFrame):
  GMT = pd.DataFrame(columns=['GAT','SOURCE', 'LAT','SLAT'])

  for x in MT.index:
    for y in MT.columns:
      for z in MT.loc[x][y]:
        GMT.loc[len(GMT)]=[x,y,z,str(y)+ '_' + z]
  return GMT

def to_GMM(GMTA:pd.DataFrame):
    df = GMTA.groupby(['GAT','SOURCE'])['LAT'].agg(list).unstack('SOURCE')
    for c in df.columns:
        df.loc[df[c].isnull(), [c]] = df.loc[df[c].isnull(), c].apply(lambda x: [])
    return df

# nuova funzione
def to_GMM(GMTA: pd.DataFrame):
    df = GMTA.groupby(['GAT', 'SOURCE'])['LAT'].agg(list).unstack('SOURCE')

    for c in df.columns:
        df[c] = df[c].apply(lambda x: [] if isinstance(x, float) and np.isnan(x) else x)

    return df

# per generare la Local Attribute Table
def generaLAT(Sources):
  LAT = pd.DataFrame(columns=['SOURCE', 'LAT', 'SLAT'])
  for x in Sources:
    for y in Sources[x].columns:
      LAT.loc[len(LAT)]=[str(x),str(y), str(x)+'_'+str(y)]
  return LAT

In [ ]:
def IsKey(R,K,Numero):
     if Numero:
          return R.groupby(K).size().reset_index(name='counts').query("counts>1").shape[0]
     else:
          return R.groupby(K).size().reset_index(name='counts').query("counts>1")


def IsFD(R,A,Det,Numero):
        if Numero:
            return R.groupby(A)[Det].nunique().reset_index(name='counts').query("counts>1").shape[0]
        else:
            return R.groupby(A)[Det].nunique().reset_index(name='counts').query("counts>1")

def ConfrontaSourceGoldStandard(_GoldStandard, _LAT):

    FOJ= pd.merge(_GoldStandard,_LAT, how='outer', indicator=True).sort_values(['GAT','SOURCE'])

    print("I seguenti LAT non sono mappati in GAT")
    print(FOJ.query(" _merge == 'right_only' ")['SLAT'].tolist())

    print("I seguenti GAT in SOURCE sono mappati in più LAT")
    print(IsFD(_GoldStandard,['GAT','SOURCE'],'LAT',0))

    print("I seguenti LAT in SOURCE sono mappati in più GAT")
    print(IsFD(_GoldStandard,['LAT','SOURCE'],'GAT',0))
    def IsKey(R,K,Numero):
     if Numero:
          return R.groupby(K).size().reset_index(name='counts').query("counts>1").shape[0]
     else:
          return R.groupby(K).size().reset_index(name='counts').query("counts>1")

# **Local attribute matching**

In [ ]:
#1 attributo alla volta
def thresholding(SimTable:pd.DataFrame, threshold:float):
    return SimTable[SimTable["sim"] > threshold].sort_values(["sim"], ascending=[False])

def top_K(SimTable:pd.DataFrame, K:int, AoB:str="A"):
    MT = deepcopy(SimTable)
    MT["pos"] = MT.sort_values(["sim"], ascending=[False]).groupby(AoB).cumcount()
    return MT[MT["pos"] < K].drop(columns=["pos"]).sort_values([AoB,"sim"], ascending=[True,False])

def top_1(SimTable:pd.DataFrame, AoB:str="A"):
    return top_K(SimTable, 1, AoB)

# **Global mapping**

In [ ]:
#global matching
#useremo queste 2 funzioni all'esame
#troviamo le corrispondenze tra schemi
def stable_marriage(MatchTable:pd.DataFrame):
    MATCH = pd.DataFrame(columns=["A", "B", "sim"])
    MT = deepcopy(MatchTable)
    MT = MT.sort_values(["sim"], ascending=[False])
    while True:
        R = MT.loc[(~MT["A"].isin(MATCH["A"])) & (~MT["B"].isin(MATCH["B"]))]
        if len(R) == 0:
            break
        x = R.iloc[0,:]
        #MATCH = MATCH.append(x, ignore_index=True)
        MATCH = pd.concat([MATCH, x], ignore_index=True)
    return MATCH

def simmetric_best_match(MatchTable:pd.DataFrame):
  CMT = deepcopy(MatchTable)

  CMT['A_RowNo'] = CMT.sort_values(['sim'], ascending=[False]) \
             .groupby(['A']) \
             .cumcount() + 1

  CMT['B_RowNo'] = CMT.sort_values(['sim'], ascending=[False]) \
             .groupby(['B']) \
             .cumcount() + 1

  return CMT[(CMT.A_RowNo==1) & (CMT.B_RowNo==1)].drop(columns=['A_RowNo', 'B_RowNo']).sort_values(['sim'], ascending=[False])

# **Funzioni di matching**

In [ ]:
#FUNZIONE con ciclo for
#il ciclo for scorre su  tutte le sorgenti
#calcolo le simtable per ogni sorgente con lo schema globale
#faccio un combiner e prendo il massimo(se vogliamo possiamo scegliere media o minimo)
#a fine ciclo facciamo l'append
def CalcoloGlobalMatchingTable(Sources:list, GlobalSchema:pd.DataFrame):
    GlobalMatchingTable = pd.DataFrame(columns=['GAT','SOURCE','LAT','SLAT','sim'])
    for y in tqdm(Sources.keys()):
    # Schema Matching tra Schema Locale Sources[y] e Schema Globale
        MatchTable=CalcoloMatchingTable(GlobalSchema, Sources[y])

     # la MatchTable tra Schema Locale Sources[y] e Schema Globale viene aggiunta alla GlobalMatchingTable

        MatchTable.columns = ['GAT','LAT','sim']
        MatchTable['SOURCE'] = str(y)
        MatchTable['SLAT'] = MatchTable['SOURCE']+'_'+MatchTable['LAT']
        #GlobalMatchingTable = GlobalMatchingTable.append(MatchTable, sort=False)
        GlobalMatchingTable = pd.concat([GlobalMatchingTable, MatchTable],sort=False)

    return GlobalMatchingTable

def CalcoloMatchingTable(TableL:pd.DataFrame,TableR:pd.DataFrame):

        SimTableA = levenshtein_label_based_similarity(TableL, TableR)
        SimTableB = jaro_label_based_similarity(TableL, TableR)
        #SimTableB = value_overlap_sim(GlobalSchema, Sources[y])
        SimTableC=value_overlap_simjoin_jaccard(TableL, TableR, 0.5)

        # combiner
        SimTable = avg_sim_table([SimTableA,SimTableB,SimTableC])
        # SimTable = min_sim_table([SimTableA,SimTableB,SimTableC])
        #SimTable = max_sim_table([SimTableA,SimTableB,SimTableC])

        # Weighted-sum
        SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [.3,.4,.3] )

        # dalla tabella di similarità alle corrispondenze

        MatchTable= thresholding(SimTable, 0.6)

        MatchTable = top_K(SimTable,2,'A')
        #MatchTable = top_K(MatchTable,2,'B')

         # global mapping
        #MatchTable = stable_marriage(MatchTable)
        #MatchTable = simmetric_best_match(MatchTable)

        return MatchTable

In [ ]:
def toA_B(Y):
  X=Y[['GAT','SLAT']].rename(columns={'GAT':'A','SLAT':'B'})
  return X

#**Includes per BabelNet**

In [ ]:
#API_KEY='f0203ed8-c34d-4d5b-a8ba-a26de7a9e63a'
API_KEY='74af617e-1aec-4415-8807-2c3e1605c802'
#API_KEY='ee15e1e1-aa53-4071-9dc9-a95449ecb37d'
#API_KEY='8520739a-7530-4c46-826a-b7ca733dd52a'
languages=['EN','ES','FR']
relation_groups=['HYPONYM','HYPERNYM','MERONYM','HOLONYM']
#forbidden_chars = ['\'','!','_',]
max_number_of_synsets=10 #se non vogliamo limiti, possiamo settarlo a 0
max_number_of_edges=100
alpha=True #filtro per valori alpha numerici
fudge_factor=0.5

In [ ]:
V = nx.Graph()

### **Funzioni per il grafo API e pathFinding**

In [ ]:
# Funzione per creare un edge con nome
def add_edge_with_label(graph, node1, node2, label, weight=None):
    """
    Aggiunge un edge tra due nodi nel grafo con un nome specifico.

    Args:
        graph: Il grafo a cui aggiungere l'edge.
        node1: Il primo nodo dell'edge.
        node2: Il secondo nodo dell'edge.
        label: Il nome da assegnare all'edge.
        weight (opzionale): Il peso dell'edge. Default è None.
    """
    graph.add_edge(node1, node2, label=label, weight=weight)

# Funzione per aggiungere un edge con proprietà
def add_edge_with_properties(graph, node1, node2, properties):
    """
    Aggiunge un edge tra due nodi nel grafo con proprietà specifiche.

    Args:
        graph: Il grafo a cui aggiungere l'edge.
        node1: Il primo nodo dell'edge.
        node2: Il secondo nodo dell'edge.
        properties: Un dizionario contenente le proprietà dell'edge.
    """
    graph.add_edge(node1, node2, **properties)
# Funzione per creare un edge
def add_edge(graph, node1, node2, weight=None):
    """
    Aggiunge un edge tra due nodi nel grafo.

    Args:
        graph: Il grafo a cui aggiungere l'edge.
        node1: Il primo nodo dell'edge.
        node2: Il secondo nodo dell'edge.
        weight (opzionale): Il peso dell'edge. Default è None.
    """
    graph.add_edge(node1, node2, weight=weight)

# Funzione per aggiungere un nodo
def add_node(graph, node):
    """
    Aggiunge un nodo al grafo.

    Args:
        graph: Il grafo a cui aggiungere il nodo.
        node: Il nodo da aggiungere.
    """
    if node not in graph:
        graph.add_node(node)
    ##else:
    ##    print("Il nodo", node, "esiste già nel grafo.")

def add_node_with_properties(graph, node, properties):
    """
    Aggiunge un nodo al grafo con proprietà specifiche.

    Args:
        graph: Il grafo a cui aggiungere il nodo.
        node: Il nodo da aggiungere.
        properties: Un dizionario contenente le proprietà del nodo.
    """
    if node not in graph:
        graph.add_node(node, **properties)
    ##else:
    ##    print("Il nodo", node, "esiste già nel grafo.")



def is_node_present(graph, node):
    """
    Aggiunge un nodo al grafo con proprietà specifiche.

    Args:
        graph: Il grafo a cui aggiungere il nodo.
        node: Il nodo da aggiungere.
        properties: Un dizionario contenente le proprietà del nodo.
    """
    if node not in graph:
        return False
    else:
        return True

def is_lemma_present_with_synsets(graph, node):
    """
    Aggiunge un nodo al grafo con proprietà specifiche.

    Args:
        graph: Il grafo a cui aggiungere il nodo.
        node: Il nodo da aggiungere.
        properties: Un dizionario contenente le proprietà del nodo.
    """
    if node not in graph:
        return False
    else:

        for neighbor in graph.neighbors(node):
        # Controlla se c'è un arco tra il nodo e il suo vicino con l'etichetta desiderata
            if 'label' in graph[node][neighbor] and graph[node][neighbor]['label'] == "HAS":
                return True
    return False

# Aggiunta di edge con proprietà
#add_edge_with_properties(G, 1, 2, {"weight": 0.5, "label": "edge1"})
#add_edge_with_properties(G, 2, 3, {"weight": 0.7, "label": "edge2"})
#add_edge_with_properties(G, 1, 3, {"weight": 0.3, "label": "edge3"})



# Aggiunta di nodi
#G.add_node(1)
#G.add_node(2)
#G.add_node(3)

# Aggiunta di edge con nomi
#add_edge_with_label(G, 1, 2, label="edge1", weight=0.5)
#add_edge_with_label(G, 2, 3, label="edge2", weight=0.7)
#add_edge_with_label(G, 1, 3, label="edge3", weight=0.3)

# Aggiunta di nodi con proprietà
#add_node_with_properties(G, 1, {"color": "red", "size": 10})
#add_node_with_properties(G, 2, {"color": "blue", "size": 15})
#add_node_with_properties(G, 3, {"color": "green", "size": 20})

# Visualizzazione delle proprietà di un nodo
#print("Proprietà del nodo 1:", G.nodes[1])

# Visualizzazione del grafo

**graphbuilding**

In [ ]:
##GRAPH BUILDING
###################################Da testare####################################################
def fetch_synset_ids_for_graph(graph,word,lemma=None,POS=None,language=None):
    # URL dell'API di BabelNet per la ricerca di un termine
    ##################################
    #controllo se word/lemma e synset sono presenti nel grafo

    if lemma is not None:
        if is_lemma_present_with_synsets(graph, lemma):
          return []
    else:
        if is_lemma_present_with_synsets(graph, word):
          return []



    ##################################
    url = f"https://babelnet.io/v9/getSynsetIds"

    # Parametri della richiesta
    params = {
        #'lemma': word,
        #'searchLang': language,  # Lingua italiana
        'key': API_KEY,  # Inserisci la tua chiave API di BabelNet qui
        #'targetLang':target
    }
    if lemma is not None:
       params['lemma']=lemma
    else:
        params['lemma']=word

    if language is not None:
       params['searchLang'] = language
    else:
       params['searchLang'] = languages
    if POS is not None:
       params['POS']=POS
    # Fai la richiesta HTTP
    response = requests.get(url, params=params)
    #response = requests.get(url)
    # Estrai il synset ID dalla risposta JSON
    synset_ids = response.json()
    #print(f"parametri {params}")
    print("risposta dell'API")
    print(response.json())
    if response.status_code != 200:
       synset_ids=[]

    #if synset is not None:
    #   synset_ids = [item for item in synset_ids if item['id'] == synset]
    if len(synset_ids)>max_number_of_synsets:
          #print(len(synset_ids))
          synset_ids=synset_ids[:max_number_of_synsets]
    #print(len(synset_ids))
    return synset_ids

#def fetch_word_from_synset_for_graph():






def get_outgoing_edges(synset_id):
    # URL dell'API di BabelNet per ottenere le relazioni uscenti da un synset
    url = f"https://babelnet.io/v9/getOutgoingEdges?id={synset_id}&key={API_KEY}"
    # Fai la richiesta HTTP
    response = requests.get(url)
    if response.status_code != 200:
       return []
    # Estrai le relazioni dalla risposta JSON
    edges = response.json()
    return edges

# controllo se la parola ha un synset annotato
# controllo se la parola è salvata nel grafo
# fetch dei synsets
# controllo se ha edges
# fetch degli edge delle parole


def add_outgoing_edges(graph,synset):
            edges = get_outgoing_edges(synset)
            if len(edges)>max_number_of_edges:
               edges=edges[:max_number_of_edges]
            for edge in edges:
                if edge['pointer']['relationGroup'] in relation_groups:
                  add_node(graph, edge['target'])
                  add_edge_with_label(graph, edge['target'], synset, edge['pointer']['relationGroup'], weight=None)


#inserisce il nodo
#inserisce il synset
#li collega
#e aggiungie gli outgoing edges del synset
def link_word_to_synset(graph,word,synset):
    add_node(graph, word)
    add_node(graph, synset)
    add_edge_with_label(graph, word, synset, "HAS", weight=None)
    add_outgoing_edges(graph,synset)

#il nostro obiettivo è di non fare fetch inutili
def add_word_to_graph(word,synset=None,lemma=None,POS=None,language=None):
        synsets=[]

        #controllo se la parola esiste nel grafo
        #vedo se ha synsets
        #vedo se ha il synset dell'annotazione
        #se non ce l'ha colleghiamo
        #altrimenti ritorniamo tutto

        if synset is not None: # controllo se il synset è presente, evitiamo la chiamata all'API
                  if lemma is not None:
                      link_word_to_synset(V,lemma,synset)
                  else:
                      link_word_to_synset(V,word,synset)

        else:#synset non presente
              #facciamo il fetch dall'api
              #inserisce già il lemma/word
              synsets=fetch_synset_ids_for_graph(V,word,lemma,POS,language)
              if len(synsets)>0:
                for synset in synsets:
                        if lemma is not None:
                            link_word_to_synset(V,lemma,synset["id"])
                        else:
                            link_word_to_synset(V,word,synset["id"])
                        #add_outgoing_edges(V,synset)
                      #link synset to word
                      #fetch degli outgoing edges
              else:
                  print("non abbiamo ricavato synsets :(")




def exctract_annotations(annotations):
    return annotations.get('synset'),annotations.get('lemma'),annotations.get('POS'),annotations.get('language')


def fetch_cols_from_dataframe(df:pd.DataFrame):
    for col in df.columns:
            if hasattr(df[col], 'annotations'): #contollo annotazioni
                #estrazione delle annotazioni
                synset_value,lemma_value,POS_value,language_value = exctract_annotations(df[col].annotations)

                if alpha: #se controlla stringa è true la parola è valida
                   if controlla_stringa(col):
                      add_word_to_graph(col,synset_value,lemma_value,POS_value,language_value)
                   elif synset_value or lemma_value is not None: #automatizzata ma con un lemma
                      add_word_to_graph(col,synset_value,lemma_value,POS_value,language_value)
                else: #aggiungiamo normalmente
                  add_word_to_graph(col,synset_value,lemma_value,POS_value,language_value)
            else: #nessuna annotazione
                if alpha and controlla_stringa(col):
                      add_word_to_graph(col)
                elif not alpha:
                      add_word_to_graph(col)

def build_graph_list(Sources:list, GlobalSchema:pd.DataFrame=None):
    for df in Sources:
        print(df.columns)
        fetch_cols_from_dataframe(df)
    if GlobalSchema is not None:
        fetch_cols_from_dataframe(GlobalSchema)


def build_graph_dictionary(Sources:list, GlobalSchema:pd.DataFrame=None):
    for df in Sources.values():
        print(df.columns)
        fetch_cols_from_dataframe(df)
    if GlobalSchema is not None:
        fetch_cols_from_dataframe(GlobalSchema)

#############################################################################################################

**Pathfinding**

In [ ]:
def find_paths_within_steps(node1, node2, max_steps):
    """
    Trova tutti i percorsi semplici tra due nodi nel grafo entro un certo numero di passi.

    Args:
        node1: Il nodo di partenza.
        node2: Il nodo di destinazione.
        max_steps: Il numero massimo di passi per il percorso.

    Returns:
        Una lista di tutti i percorsi trovati.
    """
    if not V.has_node(node1):
                  return False
    if not V.has_node(node2):
                  return False
    paths = list(nx.all_simple_paths(V, source=node1, target=node2, cutoff=max_steps))
    if len(paths)>0:
       return True

    return False

def find_paths_within_steps(node1, node2, max_steps, approved_edges=None):
    """
    Trova tutti i percorsi semplici tra due nodi nel grafo entro un certo numero di passi.

    Args:
        node1: Il nodo di partenza.
        node2: Il nodo di destinazione.
        max_steps: Il numero massimo di passi per il percorso.
        approved_edges: Una lista di edges approvati. Se non specificato, verranno considerati tutti gli edges.

    Returns:
        Una lista di tutti i percorsi trovati con le tipologie di edges.
    """
    if not V.has_node(node1) or not V.has_node(node2):
        return False

    paths = list(nx.all_simple_paths(V, source=node1, target=node2, cutoff=max_steps))
    paths_with_edge_types = []

    for path in paths:
        if approved_edges:
            edge_types = [V[u][v]['label'] for u, v in zip(path[:-1], path[1:]) if (u, v) in approved_edges]
        else:
            edge_types = [V[u][v]['label'] for u, v in zip(path[:-1], path[1:])]
        paths_with_edge_types.append((path, edge_types))

    if len(paths_with_edge_types)>0:
       return True

    return False



def find_common_nodes(node1, node2, edge_type="HAS"):
    """
    Trova i nodi collegati in comune tra due nodi tramite un certo tipo di edge.

    Args:
        node1: Il primo nodo.
        node2: Il secondo nodo.
        edge_type: Il tipo di edge da considerare. Default è "HAS".

    Returns:
        Una lista dei nodi collegati in comune, se presenti.
    """
    if not V.has_node(node1):
                  #print(f"il nodo {node1} non c'è!")
                  return False
    if not V.has_node(node2):
                  #print(f"il nodo {node2} non c'è!")
                  return False
    # Trova i vicini del primo nodo
    neighbors_node1 = set(V.neighbors(node1))
    # Trova i vicini del secondo nodo
    neighbors_node2 = set(V.neighbors(node2))
    # Trova i nodi collegati in comune
    common_nodes = neighbors_node1.intersection(neighbors_node2)
    # Filtra i nodi per il tipo di edge
    #common_nodes = [node for node in common_nodes if edge_type in graph[node1][str(node)] and edge_type in graph[node2][str(node)]]
    if len(common_nodes)>0:
       return True

    return False

**Matching functions**

In [ ]:
#######funzioni per far funzionare le cose
#convertiamo una sim matrix a sim table
#come passiamo le annotazioni?

def annotated_sim_table(TableA: pd.DataFrame, TableB: pd.DataFrame):
    # Creazione di DataFrame contenenti solo le annotazioni "lemma" delle colonne di TableA e TableB
    A_columns = []
    A_lemmas = []
    B_columns = []
    B_lemmas = []

    for col in TableA.columns:
        if hasattr(TableA[col], 'annotations'):
                annotations=TableA[col].annotations
                lemma = TableA[col].annotations.get('lemma', col)
                if lemma is not None:
                   A_columns.append(lemma)
                else:
                   A_columns.append(col)
        else:
                   A_columns.append(col)

    for col in TableB.columns:
        if hasattr(TableB[col], 'annotations'):
                annotations=TableB[col].annotations
                lemma = TableB[col].annotations.get('lemma', col)
                if lemma is not None:
                   B_columns.append(lemma)
                else:
                   B_columns.append(col)
        else:
                   B_columns.append(col)


    A = pd.DataFrame({"A": A_columns})
    B = pd.DataFrame({"B": B_columns})

    S = A.merge(B, how='cross')
    return S

similarity score functions

In [ ]:
###Graph Matching
#relation_groups=['HYPONYM','HYPERNYM','MERONYM','HOLONYM']
#1, 0.8,0.5,0
def babelnet_sim_score(searchWord,targetWord):
    if not is_node_present(V, searchWord) or not is_node_present(V, searchWord):
       return fudge_factor
    if find_common_nodes(searchWord, targetWord):
       return 1
    else:
       if find_paths_within_steps(searchWord, targetWord, 3, approved_edges=['HAS','HYPONYM','HYPERNYM']):
          return 0.8
       if find_paths_within_steps(searchWord, targetWord, 3, approved_edges=['HAS','MERONYM','HOLONYM']):
          return 0.5
    return 0


def babelnet_sim(row:pd.Series):
    return babelnet_sim_score(
            row["A"],
            row["B"]
        )

def babelnet_label_based_similarity(TableA:pd.DataFrame,TableB:pd.DataFrame):
    #fin qua le annotazioni ci sono
    C = annotated_sim_table(TableA, TableB)
    C["sim"] = C.apply(babelnet_sim, axis=1)
    return C

In [ ]:
#######funzioni per far funzionare le cose
#convertiamo una sim matrix a sim table
#come passiamo le annotazioni?

def annotated_sim_table(TableA: pd.DataFrame, TableB: pd.DataFrame):
    # Creazione di DataFrame contenenti solo le annotazioni "lemma" delle colonne di TableA e TableB
    A_columns = []
    A_lemmas = []
    B_columns = []
    B_lemmas = []

    for col in TableA.columns:
        if hasattr(TableA[col], 'annotations'):
                annotations=TableA[col].annotations
                lemma = TableA[col].annotations.get('lemma', col)
                if lemma is not None:
                   A_lemmas.append(lemma)
                else:
                   A_lemmas.append(col)
        else:
                   A_lemmas.append(col)
        A_columns.append(col)

    for col in TableB.columns:
        if hasattr(TableB[col], 'annotations'):
                annotations=TableB[col].annotations
                lemma = TableB[col].annotations.get('lemma', col)
                #B_columns.append(col)
                if lemma is not None:
                   B_lemmas.append(lemma)
                else:
                   B_lemmas.append(col)
        else:
                   B_lemmas.append(col)
        B_columns.append(col)


    A = pd.DataFrame({"A": A_columns, "A_lemma":A_lemmas})
    B = pd.DataFrame({"B": B_columns, "B_lemma":B_lemmas})

    S = A.merge(B, how='cross')
    return S

def babelnet_sim(row:pd.Series):
    sim_table = babelnet_sim_score(
            row["A_lemma"],
            row["B_lemma"]
        )
    #print(sim_table)
    #sim_table.drop(columns=['A_lemma', 'B_lemma'], inplace=True)
    return sim_table



def babelnet_label_based_similarity(TableA:pd.DataFrame,TableB:pd.DataFrame):
    #fin qua le annotazioni ci sono
    C = annotated_sim_table(TableA, TableB)
    C["sim"] = C.apply(babelnet_sim, axis=1)
    C.drop(columns=['A_lemma','B_lemma'], inplace=True)
    return C

### **Annotazione del dataframe e filtro parole automatizzate**

In [ ]:
'''column_annotations = {
        'A': {'lemma': 'lem_A', 'POS': 'noun', 'synset': 'synset_A'},
        'B': {'lemma': 'lem_B', 'POS': 'verb', 'synset': 'synset_B'},
        'C': {'lemma': 'lem_C', 'POS': 'adjective', 'synset': 'synset_C'}
    }'''

def add_column_annotations(df,column_annotations):
    for col in df.columns:
        if col in column_annotations:
            df[col].annotations = column_annotations[col]
        else:
            df[col].annotations = {}  # Se non ci sono annotazioni, impostiamo un dizionario vuoto
    return df

In [ ]:
import re

def controlla_stringa(stringa):
    # Definisci l'espressione regolare per controllare se la stringa contiene solo lettere e spazi
    regex = r'^[a-zA-ZÀ-ÿ ]+$'

    # Verifica se la stringa corrisponde all'espressione regolare
    if re.match(regex, stringa):
        return True
    else:
        return False


# **Dataset example **

In [ ]:
#Abbiamo le nostre fonti
SOURCES={}

# DataFrame in inglese
df_english = pd.DataFrame({
    'Name': ['John', 'Alice', 'Bob', 'Emily', 'Michael'],
    'Age': [25, 30, 35, 28, 40],
    'Gender': ['Male', 'Female', 'Male', 'Female', 'Male'],
    'City': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Miami'],
    'Occupation': ['Engineer', 'Doctor', 'Teacher', 'Artist', 'Lawyer']
})

# DataFrame in spagnolo
df_spanish = pd.DataFrame({
    'Nombre': ['Juan', 'Alicia', 'Roberto', 'Emilia', 'Miguel'],
    'Edad': [25, 30, 35, 28, 40],
    'Género': ['Masculino', 'Femenino', 'Masculino', 'Femenino', 'Masculino'],
    'Ciudad': ['Nueva York', 'Los Ángeles', 'Chicago', 'Houston', 'Miami'],
    'Profesión': ['Ingeniero', 'Médico', 'Maestro', 'Artista', 'Abogado']
})

# DataFrame in francese
df_french = pd.DataFrame({
    'Nom': ['Jean', 'Alice', 'Robert', 'Émilie', 'Michel'],
    'Âge': [25, 30, 35, 28, 40],
    'Sexe': ['Homme', 'Femme', 'Homme', 'Femme', 'Homme'],
    'Ville': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Miami'],
    'Profession': ['Ingénieur', 'Médecin', 'Enseignant', 'Artiste', 'Avocat']
})


SOURCES['S1']=df_english.astype(str)
SOURCES['S2']=df_spanish.astype(str)
#SOURCES['S3']=df_french.astype(str)
GlobalSchema=df_french.astype(str)
LAT = pd.DataFrame(columns=['SOURCE', 'LAT', 'SLAT'])
for x in SOURCES.keys():
    for y in SOURCES[x].columns:
      LAT.loc[len(LAT)]=[str(x),str(y), str(x)+'_'+str(y)]


S1_annotations = {
          'Name': {'language': 'EN'},
          'Age': {'language': 'EN'},
          'Gender':{'language': 'EN'},
          'City': {'language': 'EN'},
          'Occupation': {'language': 'EN'},
    }

S2_annotations = {
    'Nombre': {'language': 'ES'},
    'Edad': {'language': 'ES'},
    'Género': {'language': 'ES'},
    'Ciudad': {'language': 'ES'},
    'Profesión':{ 'synset':'bn:00064608n','language': 'ES'}
}

S3_annotations = {
      'Nom':  {'language': 'FR'},
      'Âge': {'language': 'FR'},
      'Sexe': {'language': 'FR'},
      'Ville':{'language': 'FR'},
      'Profession': {'language': 'FR'}
}

SOURCES['S1']=add_column_annotations(SOURCES['S1'],S1_annotations)
SOURCES['S2']=add_column_annotations(SOURCES['S2'],S2_annotations)
GlobalSchema=add_column_annotations(GlobalSchema,S3_annotations)

GoldStandard = pd.read_csv("gold_standard_td.csv",sep=',')
ListOfGAT=GoldStandard['GAT'].drop_duplicates().to_list()


build_graph_dictionary(SOURCES,GlobalSchema)

Index(['Name', 'Age', 'Gender', 'City', 'Occupation'], dtype='object')
risposta dell'API
[{'id': 'bn:00064751n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:01454938n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00037747n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00056758n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:01765022n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:27426992n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:03508754n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:24176884n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00034390n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00045822n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00086766v', 'pos': 'VERB', 'source': 'BABELNET'}, {'id': 'bn:00086663v', 'pos': 'VERB', 'source': 'BABELNET'}, {'id': 'bn:07958411n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00084385v', 'pos': 'VERB', 'source': 'BABELNET'}, {'id': 'bn:00090560v', 'pos': 'VERB', 'source': 'BABELNE

In [ ]:
#noi dobbiamo solo cambiare il metodo CalcoloMatchingTable
#e capire come includere le sources e le annotazioni


#Difeniamo la funzione per la matching Table
def CalcoloMatchingTable(TableL:pd.DataFrame,TableR:pd.DataFrame):
        SimTableA = levenshtein_label_based_similarity(TableL, TableR)
        #SimTableA = value_overlap_extended_jaccard_LEV(TableL, TableR,0.55)
        #SimTableB = jaro_label_based_similarity(TableL, TableR)
        SimTableB = babelnet_label_based_similarity(TableL,TableR)
        #SimTableB = value_overlap_sim(GlobalSchema, Sources[y])
        SimTableC=  value_overlap_simjoin_jaccard(TableL, TableR, 0.6)

        # combiner
        #SimTable = avg_sim_table([SimTableA,SimTableB,SimTableC])
        # SimTable = min_sim_table([SimTableA,SimTableB,SimTableC])
        #SimTable = max_sim_table([SimTableA,SimTableB,SimTableC])

        # Weighted-sum
        SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.4,0.3,0.3] )

        # dalla tabella di similarità alle corrispondenze

        MatchTable= thresholding(SimTable, 0.40)

        #MatchTable = top_K(SimTable,'A',2)
        MatchTable = top_1(MatchTable,'A')

         # global mapping
        #MatchTable = stable_marriage(MatchTable)
        #MatchTable = simmetric_best_match(MatchTable)
        return MatchTable

In [ ]:
Valuta(toA_B(GoldStandard),toA_B(CalcoloGlobalMatchingTable(SOURCES, GlobalSchema))).rename(index={0: 'BabelNet label based'})


100%|██████████| 2/2 [00:01<00:00,  1.63it/s]


,MT,TP,FP,FN,P,R,F
BabelNet label based,7,7,0,3,1.0,0.7,0.8235


In [ ]:
Valuta(toA_B(GoldStandard),toA_B(CalcoloGlobalMatchingTable(SOURCES, GlobalSchema))).rename(index={0: 'BabelNet label based'})

100%|██████████| 2/2 [00:01<00:00,  1.43it/s]


,MT,TP,FP,FN,P,R,F
BabelNet label based,6,6,0,4,1.0,0.6,0.75


In [ ]:
Valuta(toA_B(GoldStandard),toA_B(CalcoloGlobalMatchingTable(SOURCES, GlobalSchema))).rename(index={0: 'BabelNet label based'})


100%|██████████| 2/2 [00:00<00:00,  2.69it/s]


,MT,TP,FP,FN,P,R,F
BabelNet label based,9,9,0,1,1.0,0.9,0.9474


In [ ]:
Valuta(toA_B(GoldStandard),toA_B(CalcoloGlobalMatchingTable(SOURCES, EmptyGlobalSchema))).rename(index={0: 'BabelNet label based'})


# **Prova con BabelNet**

In [ ]:
# ESEMPIO
# Instanced Global Schema
GlobalSchema=pd.read_csv("https://dbgroup.ing.unimore.it/SIWS/E1/GlobalSchema.csv").astype(str)

# Empty Global Schema : è lo stesso schema senza istanze
EmptyGlobalSchema=GlobalSchema[0:0]
EmptyGlobalSchema

,rec_id,Name,Surname,Main Work Address (City),Main Work Address (Phone),Secondary_Work_Address_City_Phone,Phone4City3,Other Phone,Age1,Age2,Age4


In [ ]:
#abbiamo 3 sorgenti
#li mettiamo in un dizionario
src_links = [
 'https://dbgroup.ing.unimore.it/SIWS/E1/S1.csv',
 'https://dbgroup.ing.unimore.it/SIWS/E1/S2.csv',
 'https://dbgroup.ing.unimore.it/SIWS/E1/S3.csv']

SOURCES = { 'S'+str(i+1) : pd.read_csv(link).astype(str) for i, link in enumerate(src_links) }

data = [[name,col, name+'_'+col] for name, df in SOURCES.items() for col in df.columns]
LAT = pd.DataFrame(columns=['SOURCE', 'LAT', 'SLAT'],data=data)
build_graph_dictionary(SOURCES,EmptyGlobalSchema)

Index(['rec_id', 'Name1', 'Name2', 'City1', 'Phone2', 'Phone3', 'Phone4',
       'Age1', 'Age2', 'Sex', 'gender2', 'X1', 'X2'],
      dtype='object')
risposta dell'API
[{'id': 'bn:16377301n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:15789175n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:27545715n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:21095326n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:06248459n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:03634433n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:02183058n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:02174854n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:10943110n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:11457069n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00070772n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:01022274n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00016124n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:02178921n', 'pos': 'NOUN', 

In [ ]:
#applichiamo lo stesso metodo visto sopra
TableB=GlobalSchema
TableA=SOURCES['S1']
SimLabelLEV = babelnet_label_based_similarity(TableA, TableB)
SimLabelLEV

il nodo rec_id non c'è!
il nodo Main Work Address (City) non c'è!
il nodo Main Work Address (Phone) non c'è!
il nodo Secondary_Work_Address_City_Phone non c'è!
il nodo Phone4City3 non c'è!
il nodo Other Phone non c'è!
il nodo Age1 non c'è!
il nodo Age2 non c'è!
il nodo Age4 non c'è!


,A,B,sim
0,rec_id,rec_id,0.3
1,rec_id,Name,0.3
2,rec_id,Surname,0.3
3,rec_id,Main Work Address (City),0.3
4,rec_id,Main Work Address (Phone),0.3
...,...,...,...
138,X2,Phone4City3,0.3
139,X2,Other Phone,0.3
140,X2,Age1,0.3
141,X2,Age2,0.3


In [ ]:
def CalcoloMatchingTable(TableL:pd.DataFrame,TableR:pd.DataFrame):

        SimTableA = levenshtein_label_based_similarity(TableL, TableR)
        SimTableB = jaro_label_based_similarity(TableL, TableR)
        #SimTableB = value_overlap_sim(GlobalSchema, Sources[y])
        #SimTableC=value_overlap_simjoin_jaccard(TableL, TableR, 0.5)
        SimTableC=babelnet_label_based_similarity(TableL, TableR)
        # combiner
        SimTable = avg_sim_table([SimTableA,SimTableB,SimTableC])
        # SimTable = min_sim_table([SimTableA,SimTableB,SimTableC])
        #SimTable = max_sim_table([SimTableA,SimTableB,SimTableC])

        # Weighted-sum
        SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [.3,.4,.3] )

        # dalla tabella di similarità alle corrispondenze

        MatchTable= thresholding(SimTable, 0.6)

        MatchTable = top_K(SimTable,2,'A')
        #MatchTable = top_K(MatchTable,2,'B')

         # global mapping
        #MatchTable = stable_marriage(MatchTable)
        #MatchTable = simmetric_best_match(MatchTable)

        return MatchTable

In [ ]:
#applichiamo lo stesso metodo visto sopra
TableB=GlobalSchema
TableA=SOURCES['S1']
SimLabelLEV = levenshtein_label_based_similarity(TableA, TableB)
SimLabelLEV

In [ ]:
SimLabelJW = JaroWinkler_label_based_similarity(TableA, TableB)
SimLabelJW

In [ ]:
SimOverlapGEN = generalized_value_overlap_sim(TableA, TableB, 0.3)
SimOverlapGEN.sort_values('sim', ascending=False).head(3)

In [ ]:
#unisco le diverse tabelle di similarità e ottengo la media delle coppie
avg_sim_table([SimLabelLEV,SimLabelJW,SimOverlapGEN])

In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)

 67%|██████▋   | 2/3 [00:00<00:00, 13.82it/s]

il nodo rec_id non c'è!
il nodo Name1 non c'è!
il nodo Name2 non c'è!
il nodo City1 non c'è!
il nodo Phone2 non c'è!
il nodo Phone3 non c'è!
il nodo Phone4 non c'è!
il nodo Age1 non c'è!
il nodo Age2 non c'è!
il nodo gender2 non c'è!
il nodo X1 non c'è!
il nodo X2 non c'è!
il nodo rec_id non c'è!
il nodo Name1 non c'è!
il nodo Name2 non c'è!
il nodo City1 non c'è!
il nodo Phone2 non c'è!
il nodo Phone3 non c'è!
il nodo Phone4 non c'è!
il nodo Age1 non c'è!
il nodo Age2 non c'è!
il nodo gender2 non c'è!
il nodo X1 non c'è!
il nodo X2 non c'è!
il nodo rec_id non c'è!
il nodo Nome Completo non c'è!
il nodo City2 non c'è!
il nodo Phone1 non c'è!
il nodo Age1 non c'è!
il nodo Age3 non c'è!
il nodo City3 non c'è!
il nodo Sex2 non c'è!
il nodo gender2 non c'è!
il nodo X1 non c'è!
il nodo X3 non c'è!
il nodo rec_id non c'è!
il nodo Nome Completo non c'è!
il nodo City2 non c'è!
il nodo Phone1 non c'è!
il nodo Age1 non c'è!
il nodo Age3 non c'è!
il nodo City3 non c'è!
il nodo Sex2 non c'è!
il no

100%|██████████| 3/3 [00:00<00:00, 13.11it/s]


In [ ]:
to_GMM(GMT)

In [ ]:
#Carichiamo il goldstandard
GoldStandard=pd.read_csv("https://dbgroup.ing.unimore.it/SIWS/E1/GoldStandardEsempioE1.csv").astype(str)

In [ ]:
to_GMM(GoldStandard)

SOURCE,S1,S2,S3
GAT,,,
Age1,[Age1],[Age1],[]
Age2,[Age2],[],[Age2]
Age4,[],[],[Age4]
Main Work Address (City),[City],[],[City1]
Main Work Address (Phone),[],[Phone1],[Phone1]
Name,[Name],[Full Name],[Full Name]
Other Phone,[Phone3],[],[Phone3]
Phone4City3,[Phone4],[City3],"[City3, Phone4]"
Secondary_Work_Address_City_Phone,[Phone2],[City2],[City2]


In [ ]:
GoldStandard[['GAT', 'SLAT']].head(5)

,GAT,SLAT
0,rec_id,S1_rec_id
1,rec_id,S2_rec_id
2,rec_id,S3_rec_id
3,Name,S1_Name1
4,Name,S2_Nome Completo


In [ ]:
#calcoliamo precisione recall
GMT=CalcoloGlobalMatchingTable(SOURCES,EmptyGlobalSchema)
Valuta(toA_B(GoldStandard),toA_B(GMT))

100%|██████████| 3/3 [00:00<00:00, 13.77it/s]

il nodo rec_id non c'è!
il nodo Name1 non c'è!
il nodo Name2 non c'è!
il nodo City1 non c'è!
il nodo Phone2 non c'è!
il nodo Phone3 non c'è!
il nodo Phone4 non c'è!
il nodo Age1 non c'è!
il nodo Age2 non c'è!
il nodo gender2 non c'è!
il nodo X1 non c'è!
il nodo X2 non c'è!
il nodo rec_id non c'è!
il nodo Name1 non c'è!
il nodo Name2 non c'è!
il nodo City1 non c'è!
il nodo Phone2 non c'è!
il nodo Phone3 non c'è!
il nodo Phone4 non c'è!
il nodo Age1 non c'è!
il nodo Age2 non c'è!
il nodo gender2 non c'è!
il nodo X1 non c'è!
il nodo X2 non c'è!
il nodo rec_id non c'è!
il nodo Nome Completo non c'è!
il nodo City2 non c'è!
il nodo Phone1 non c'è!
il nodo Age1 non c'è!
il nodo Age3 non c'è!
il nodo City3 non c'è!
il nodo Sex2 non c'è!
il nodo gender2 non c'è!
il nodo X1 non c'è!
il nodo X3 non c'è!
il nodo rec_id non c'è!
il nodo Nome Completo non c'è!
il nodo City2 non c'è!
il nodo Phone1 non c'è!
il nodo Age1 non c'è!
il nodo Age3 non c'è!
il nodo City3 non c'è!
il nodo Sex2 non c'è!
il no

,MT,TP,FP,FN,P,R,F
0,66,15,51,12,0.2273,0.5556,0.3226


In [ ]:
Vedi_Valuta(toA_B(GoldStandard),toA_B(GMT),'FP')

In [ ]:
# dataframe per memorizzare le valutazioni dei vai metodi
ValutazioneMatchTable = pd.DataFrame(columns=['MT', 'TP', 'FP', 'FN', 'P', 'R', 'F'])

In [ ]:
ValutazioneMatchTable = ValutazioneMatchTable.append(
    Valuta(toA_B(GoldStandard),toA_B(GMT)).rename(index={0: 'ProvaConAvg'}))
ValutazioneMatchTable

# **Esercizio Movie Dataset**

In [ ]:
src_links = [
'http://pages.cs.wisc.edu/~anhai/data/784_data/movies5/csv_files/imdb.csv',
'http://pages.cs.wisc.edu/~anhai/data/784_data/movies5/csv_files/roger_ebert.csv',
'http://pages.cs.wisc.edu/~anhai/data/784_data/movies1/csv_files/rotten_tomatoes.csv']


S1_annotations = {
          'movie_name': {'lemma':'title','language': 'EN'},
          'movie_rating': {'lemma':'rating','language': 'EN'},
    }

S2_annotations = {
          'movie_name': {'lemma':'title','language': 'EN'},
          'critic_rating': {'lemma':'rating','language': 'EN'},
          'pg_rating':{'lemma':'PEGI','language':'EN'}
}

S3_annotations = {
          'movie_name': {'lemma':'title','language': 'EN'},
          'movie_rating': {'lemma':'rating','language': 'EN'},
}
SOURCES = { str(i) : pd.read_csv(link).astype(str) for i, link in enumerate(src_links) }
SOURCES['0']=add_column_annotations(SOURCES['0'],S1_annotations)
SOURCES['1']=add_column_annotations(SOURCES['1'],S2_annotations)
SOURCES['2']=add_column_annotations(SOURCES['2'],S3_annotations)

build_graph_dictionary(SOURCES)

Index(['id', 'movie_name', 'year', 'directors', 'actors', 'movie_rating',
       'genre', 'duration'],
      dtype='object')
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
risposta dell'API
[]
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
Index(['id', 'movie_name', 'year', 'directors', 'actors', 'critic_rating',
       'genre', 'pg_rating', 'duration'],
      dtype='object')
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
risposta dell'API
[]
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
Index(['Id', 'Name', 'Year', 'Release Date', 'Director', 'Creator', 'Actors',
       'Cast', 'Language', 'Country', 'Duration', 'RatingValue', 'Rating

In [ ]:
SOURCES['0'].head()

,id,movie_name,year,directors,actors,movie_rating,genre,duration
0,0,High-Rise,2015,Ben Wheatley,"Tom Hiddleston, Jeremy Irons, Sienna Miller",6.8,"Action, Drama, Sci-Fi",112 min
1,1,Mercy for Angels,2015,K.C. Amos,"Vida Guerra, Emilio Rivera, John Amos",6.2,"Action, Drama, Thriller",91 min
2,2,Wind Walkers,2015,Russell Friedenberg,"Glen Powell, Zane Holtz, Rudy Youngblood",5.6,"Action, Horror, Thriller",93 min
3,3,Alcatraz Prison Escape: Deathbed Confession,2015,John Edward Lee,"Danny Trejo, Ed O'Ross, Shelby Deekins",6.6,"Action, Crime",93 min
4,4,Barely Lethal,2015,Kyle Newman,"Jaime King, Samuel L. Jackson, Madeleine Stack",5.3,"Action, Adventure, Comedy",96 min


In [ ]:
import pandas as pd
GoldStandard=pd.read_csv('http://dbgroup.ing.unimore.it/SIWS/DataIntegration/Movie_MTGS.csv')
GoldStandard['SOURCE']=GoldStandard['SOURCE'].apply(str)
#to_GMM(GoldStandard)
GoldStandard.head(15)

,GAT,SOURCE,LAT,SLAT
0,Filming_Locations,2,Filming Locations,2_Filming Locations
1,RatingCount,2,RatingCount,2_RatingCount
2,ReviewCount,2,ReviewCount,2_ReviewCount
3,movie_country,2,Country,2_Country
4,movie_genre,0,genre,0_genre
5,movie_genre,1,genre,1_genre
6,movie_genre,2,Genre,2_Genre
7,movie_genre,3,Genre,3_Genre
8,movie_genre,5,Genre,5_Genre
9,movie_genre,6,Genre,6_Genre


In [ ]:
ListOfGAT=GoldStandard['GAT'].drop_duplicates().to_list()
ListOfGAT

['Filming_Locations',
 'RatingCount',
 'ReviewCount',
 'movie_country',
 'movie_genre',
 'movie_duration',
 'movie_ContentRating',
 'movie_year',
 'movie_rating',
 'Id',
 'movie_title',
 'movie_date',
 'movie_director',
 'movie_creators',
 'movie_cast',
 'movie_language',
 'movie_url',
 'movie_time',
 'movie_cost',
 'movie_summary']

In [ ]:
#
# Simuliamo uno schema globale vuoto
#
ListOfGAT=GoldStandard['GAT'].drop_duplicates().to_list()
EmptyGlobalSchema = pd.DataFrame(columns=ListOfGAT)
global_annotations = {
          'Filming_Locations': {'lemma':'location','language': 'EN'},
          'movie_genre': {'lemma':'genre','language': 'EN'},
          'movie_duration': {'lemma':'duration','language': 'EN'},
          'movie_ContentRating': {'lemma':'PEGI','language': 'EN'},
          'movie_year': {'lemma':'year','language': 'EN'},
          'movie_rating': {'lemma':'rating','language': 'EN'},
          'movie_title': {'lemma':'title','language': 'EN'},
          'movie_date': {'lemma':'date','language': 'EN'},
          'movie_director': {'lemma':'director','language': 'EN'},
          'movie_cast': {'lemma':'cast','language': 'EN'},
          'movie_language': {'lemma':'language','language': 'EN'},

}
EmptyGlobalSchema=add_column_annotations(EmptyGlobalSchema,global_annotations)
build_graph_dictionary(SOURCES,EmptyGlobalSchema)

Index(['id', 'movie_name', 'year', 'directors', 'actors', 'movie_rating',
       'genre', 'duration'],
      dtype='object')
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
risposta dell'API
[]
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
Index(['id', 'movie_name', 'year', 'directors', 'actors', 'critic_rating',
       'genre', 'pg_rating', 'duration'],
      dtype='object')
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
risposta dell'API
[]
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
non abbiamo ricavato synsets :(
Index(['Id', 'Name', 'Year', 'Release Date', 'Director', 'Creator', 'Actors',
       'Cast', 'Language', 'Country', 'Duration', 'RatingValue', 'Rating

In [ ]:
def CalcoloMatchingTable(TableL:pd.DataFrame,TableR:pd.DataFrame):

        SimTableA = generalized_value_overlap_sim(TableL, TableR,0.5)
        SimTableB = babelnet_label_based_similarity(TableL, TableR)
        #SimTableB = value_overlap_sim(GlobalSchema, Sources[y])
        SimTableC=value_overlap_simjoin_jaccard(TableL, TableR, 0.5)

        # combiner
        #SimTable = avg_sim_table([SimTableA,SimTableB,SimTableC])
        # SimTable = min_sim_table([SimTableA,SimTableB,SimTableC])
        SimTable = max_sim_table([SimTableA,SimTableB,SimTableC])

        # Weighted-sum
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [.3,.4,.3] )

        # dalla tabella di similarità alle corrispondenze

        MatchTable= thresholding(SimTable, 0.7)

        MatchTable = top_K(SimTable,2,'A')
        #MatchTable = top_K(MatchTable,2,'B')

         # global mapping
        #MatchTable = stable_marriage(MatchTable)
        #MatchTable = simmetric_best_match(MatchTable)

        return MatchTable

In [ ]:
#
# Calcoliamo dei match label based con lo schema vuoto
#
to_GMM(CalcoloGlobalMatchingTable(SOURCES, EmptyGlobalSchema))


  0%|          | 0/3 [00:00<?, ?it/s]

il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!


 33%|███▎      | 1/3 [00:14<00:29, 14.60s/it]

il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!


 67%|██████▋   | 2/3 [00:23<00:11, 11.33s/it]

il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo Rating

100%|██████████| 3/3 [01:15<00:00, 25.00s/it]


SOURCE,0,1,2
GAT,,,
Filming_Locations,"[movie_rating, directors]","[critic_rating, pg_rating]","[Filming Locations, Duration]"
Id,"[id, duration]","[id, duration]","[Id, Country]"
PEGI,"[genre, duration]","[PEGI, year]","[Duration, RatingCount]"
RatingCount,"[duration, directors]","[duration, actors]","[RatingCount, ReviewCount]"
ReviewCount,"[duration, directors]","[duration, directors]","[ReviewCount, RatingCount]"
cast,"[genre, duration]","[PEGI, duration]","[Cast, Duration]"
date,"[directors, duration]","[id, duration]","[Id, Name]"
director,"[title, year]","[id, title]","[Director, Cast]"
duration,"[duration, year]","[duration, id]","[Duration, Id]"


In [ ]:
ValutazioneMatchTable = pd.DataFrame(columns=['MT', 'TP', 'FP', 'FN', 'P', 'R', 'F'])
ValutazioneMatchTable

,MT,TP,FP,FN,P,R,F


In [ ]:
#ProvaConAvg_07_SM=CalcoloGlobalMatchingTable(SOURCES, EmptyGlobalSchema)
#ValutazioneMatchTable =
#ValutazioneMatchTable.append(
Valuta(toA_B(GoldStandard),toA_B(CalcoloGlobalMatchingTable(SOURCES, EmptyGlobalSchema))).rename(index={0: 'ProvaConAvg_07_SBM'})
    #)
#ValutazioneMatchTable

  0%|          | 0/3 [00:00<?, ?it/s]

il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!


 33%|███▎      | 1/3 [00:21<00:43, 21.73s/it]

il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!
il nodo directors non c'è!


 67%|██████▋   | 2/3 [00:35<00:17, 17.14s/it]

il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo RatingCount non c'è!
il nodo ReviewCount non c'è!
il nodo RatingValue non c'è!
il nodo Rating

100%|██████████| 3/3 [01:49<00:00, 36.60s/it]


,MT,TP,FP,FN,P,R,F
ProvaConAvg_07_SBM,186,10,176,61,0.0538,0.1408,0.0778


In [ ]:
#
#Proviamo con delle istanze
#
for x in SOURCES.keys():
  print(x, SOURCES[x].shape)

0 (6913, 8)
1 (3556, 9)
2 (7390, 17)


In [ ]:
#
# Scegliamo la source '2' perchè ha più istanze e attributi, rimuoviamola dalle fonti
#
GlobalSchema = SOURCES['2']
del SOURCES['2']

In [ ]:
#Guardando gli schemi, notiamo che c'è un match tra:
#Movie rating della fonte 1 e fonte 3
pd.merge(GoldStandard[GoldStandard.SOURCE=='2'],
         GoldStandard[GoldStandard.SOURCE=='0'], on='GAT')[['LAT_x','SOURCE_y','LAT_y','SLAT_y']]

,LAT_x,SOURCE_y,LAT_y,SLAT_y
0,Genre,0,genre,0_genre
1,Duration,0,duration,0_duration
2,Year,0,year,0_year
3,RatingValue,0,movie_rating,0_movie_rating
4,Name,0,movie_name,0_movie_name
5,Director,0,directors,0_directors
6,Actors,0,actors,0_actors
7,Cast,0,actors,0_actors


In [ ]:
GLobalStandard2 = pd.DataFrame(columns=['GAT','SOURCE','LAT','SLAT'])
from tqdm import tqdm
for y in tqdm(SOURCES.keys()):
    if (y!=2):
        X= pd.merge(GoldStandard[GoldStandard.SOURCE=='2'],
         GoldStandard[GoldStandard.SOURCE==y], on='GAT')[['LAT_x','SOURCE_y','LAT_y','SLAT_y']]
        X.columns=['GAT','SOURCE','LAT','SLAT']
        print(y,len(X))
        GLobalStandard2 = GLobalStandard2.append(X, sort=False)
GLobalStandard2

100%|██████████| 3/3 [00:00<00:00, 63.32it/s]

0 8
1 8
2 19


,GAT,SOURCE,LAT,SLAT
0,Genre,0,genre,0_genre
1,Duration,0,duration,0_duration
2,Year,0,year,0_year
3,RatingValue,0,movie_rating,0_movie_rating
4,Name,0,movie_name,0_movie_name
5,Director,0,directors,0_directors
6,Actors,0,actors,0_actors
7,Cast,0,actors,0_actors
0,Genre,1,genre,1_genre
1,Duration,1,duration,1_duration


In [ ]:
#
# Ridefiniamo di nuovo la funzione di Matching
#
def CalcoloGlobalMatchingTable(Sources:list, GlobalSchema:pd.DataFrame):
    GlobalMatchingTable = pd.DataFrame(columns=['GAT','SOURCE','LAT','SLAT','sim'])
    from tqdm import tqdm
    for y in tqdm(Sources.keys()):
        SimTableA = levenshtein_label_based_similarity(GlobalSchema, Sources[y])
#        SimTableB = jaro_label_based_similarity(GlobalSchema, Sources[y])
        #SimTableB = value_overlap_sim(GlobalSchema, Sources[y])
        SimTableC=value_overlap_simjoin_jaccard(GlobalSchema, Sources[y], 0.9)
        SimTable = avg_sim_table([SimTableA,SimTableC])
#        SimTable = max_sim_table([SimTableA,SimTableB])
#        SimTable = max_sim_table([SimTableA,SimTableB,SimTableC])

        MatchTable= thresholding(SimTable, 0.3)
        #MatchTable = stable_marriage(MatchTable)
        #MatchTable = top_K(SimTable,2)
        #MatchTable = top_K(MatchTable,2)

        MatchTable.columns = ['GAT','LAT','sim']
        MatchTable['SOURCE'] = str(y)
        MatchTable['SLAT'] = MatchTable['SOURCE']+'_'+MatchTable['LAT']
        GlobalMatchingTable = GlobalMatchingTable.append(MatchTable, sort=False)
    return GlobalMatchingTable

In [ ]:
X=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)

100%|██████████| 3/3 [02:16<00:00, 45.40s/it]


In [ ]:
Valuta(toA_B(GLobalStandard2),toA_B(X))

,MT,TP,FP,FN,P,R,F
0,34,28,6,7,0.8235,0.8,0.8116


# **Corpo esercizio**

In [ ]:
src_links = [
'http://dbgroup.ing.unimore.it/SIWS/DataIntegration/Esempi/EsempioBook/BookA1.csv',
'http://dbgroup.ing.unimore.it/SIWS/DataIntegration/Esempi/EsempioBook/BookB1.csv',
'http://dbgroup.ing.unimore.it/SIWS/DataIntegration/Esempi/EsempioBook/BookC1.csv'
]

SOURCES = { 'S'+str(i+1) : pd.read_csv(link).astype(str) for i, link in enumerate(src_links) }
GlobalSchema=pd.read_csv('http://dbgroup.ing.unimore.it/SIWS/DataIntegration/Esempi/EsempioBook/GlobalSchema.csv').astype(str)

In [ ]:
SOURCES['S1'].head()
S1_annotations = {
          'LeadAuthor': {'lemma':'lead author'},
          'NumberOfPages': {'lemma':'pages','language': 'EN'},
          'paperbackprice':{'lemma':'price','language':'EN'},
          'nookbookprice':{'lemma':'price','language':'EN'},
          'productdimensions':{'lemma':'size','language':'EN'},
          'publicationdate':{'lemma':'publication date','language':'EN'},
          'ratingvalue':{'lemma':'rating','language':'EN'},
          'BookName':{'lemma':'title','language':'EN'},
}


SOURCES['S1']=add_column_annotations(SOURCES['S1'],S1_annotations)

In [ ]:
list(SOURCES['S1'].columns)

['LeadAuthor',
 'coAuthor',
 'nookbookprice',
 'pages',
 'paperbackprice',
 'productdimensions',
 'publicationdate',
 'publisher',
 'ratingscount',
 'ratingvalue',
 'title',
 'size']

In [ ]:
SOURCES['S2'].head()
S2_annotations = {
          'FirstAuthor': {'lemma':'lead author'},
          'NumberOfPages': {'lemma':'pages','language': 'EN'},
          'paperbackprice':{'lemma':'price','language':'EN'},
          'price':{'lemma':'price','language':'EN'},
          'size':{'lemma':'size','language':'EN'},
          'dateOfIssue':{'lemma':'publication date','language':'EN'},
          'valuation':{'lemma':'rating','language':'EN'},
          'BookName':{'lemma':'title','language':'EN'},
}


SOURCES['S2']=add_column_annotations(SOURCES['S2'],S2_annotations)

In [ ]:
list(SOURCES['S2'].columns)

['FirstAuthor',
 'Autors',
 'price',
 'pages',
 'paperbackprice',
 'size',
 'dateOfIssue',
 'YearEditor',
 'valuation',
 'BookName']

In [ ]:
SOURCES['S3'].head()
S3_annotations = {
          'author1': {'lemma':'lead author'},
          'NumberOfPages': {'lemma':'pages','language': 'EN'},
          'paperbackprice':{'lemma':'price','language':'EN'},
          'widthXlength':{'lemma':'size','language':'EN'},
          'date':{'lemma':'publication date','language':'EN'},
          'assessmentvalue':{'lemma':'rating','language':'EN'},
}
SOURCES['S3']=add_column_annotations(SOURCES['S3'],S3_annotations)

In [ ]:
list(SOURCES['S3'].columns)

['author1',
 'NumberOfPages',
 'paperbackprice',
 'widthXlength',
 'date',
 'PubblicationInfo',
 'assessmentvalue',
 'title']

In [ ]:
GlobalSchema
global_annotations = {
          'author1': {'lemma':'lead author'},
          'author2': {'lemma':'author','language': 'EN'},
          'author2':{'lemma':'author','language':'EN'},
          'nookbookprice':{'lemma':'price','language':'EN'},
          'hardcoverprice':{'lemma':'price','language':'EN'},
          'paperbackprice':{'lemma':'price','language':'EN'},
          'productdimensions':{'lemma':'size','language':'EN'},
          'publicationdate':{'lemma':'publication date','language':'EN'},
          'ratingscount':{'lemma':'count','language':'EN'},
          'ratingvalue':{'lemma':'rating','language':'EN'},
}


GlobalSchema=add_column_annotations(GlobalSchema,global_annotations)

In [ ]:
list(GlobalSchema.columns)

['author1',
 'author2',
 'author3',
 'nookbookprice',
 'hardcoverprice',
 'pages',
 'paperbackprice',
 'productdimensions',
 'publicationdate',
 'publisher',
 'ratingscount',
 'ratingvalue',
 'title']

In [ ]:
build_graph_dictionary(SOURCES,GlobalSchema)

Index(['LeadAuthor', 'coAuthor', 'nookbookprice', 'pages', 'paperbackprice',
       'productdimensions', 'publicationdate', 'publisher', 'ratingscount',
       'ratingvalue', 'title', 'size'],
      dtype='object')
risposta dell'API
[{'id': 'bn:02431076n', 'pos': 'NOUN', 'source': 'BABELNET'}]
risposta dell'API
[{'id': 'bn:00020199n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:13782939v', 'pos': 'VERB', 'source': 'BABELNET'}]
risposta dell'API
[{'id': 'bn:29749519n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00092009v', 'pos': 'VERB', 'source': 'BABELNET'}, {'id': 'bn:02661333n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:03752619n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00064307n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00896133n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00925119n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:00064309n', 'pos': 'NOUN', 'source': 'BABELNET'}, {'id': 'bn:23701999n', 'pos': 'NOUN', 'source': 'BABELNET

In [ ]:
# questo è il gold standard dato
GMT_GS=pd.read_csv('http://dbgroup.ing.unimore.it/SIWS/DataIntegration/Esempi/EsempioBook/GMT_GSesempio.csv').astype(str)
to_GMM(GMT_GS)

SOURCE,S1,S2,S3
GAT,,,
author1,[LeadAuthor],[FirstAuthor],[author1]
author2,[coAuthor],[Autors],[]
author3,[],[Autors],[]
nookbookprice,"[nookbookprice, paperbackprice]","[price, paperbackprice]",[paperbackprice]
pages,"[pages, ratingscount]","[pages, valuation]","[NumberOfPages, assessmentvalue]"
paperbackprice,"[paperbackprice, nookbookprice]","[paperbackprice, price]",[paperbackprice]
productdimensions,"[productdimensions, size]",[size],[widthXlength]
publicationdate,[publicationdate],[dateOfIssue],[date]
publisher,[publisher],[],[PubblicationInfo]


In [ ]:
data = [[name,col, name+'_'+col] for name, df in SOURCES.items() for col in df.columns]
LAT = pd.DataFrame(columns=['SOURCE', 'LAT', 'SLAT'],data=data)
LAT

,SOURCE,LAT,SLAT
0,S1,LeadAuthor,S1_LeadAuthor
1,S1,coAuthor,S1_coAuthor
2,S1,nookbookprice,S1_nookbookprice
3,S1,pages,S1_pages
4,S1,paperbackprice,S1_paperbackprice
5,S1,productdimensions,S1_productdimensions
6,S1,publicationdate,S1_publicationdate
7,S1,publisher,S1_publisher
8,S1,ratingscount,S1_ratingscount
9,S1,ratingvalue,S1_ratingvalue


In [ ]:
ConfrontaSourceGoldStandard(GMT_GS, LAT)

I seguenti LAT non sono mappati in GAT
['S2_YearEditor', 'S2_BookName']
I seguenti GAT in SOURCE sono mappati in più LAT
                  GAT SOURCE  counts
6       nookbookprice     S1       2
7       nookbookprice     S2       2
9               pages     S1       2
10              pages     S2       2
11              pages     S3       2
12     paperbackprice     S1       2
13     paperbackprice     S2       2
15  productdimensions     S1       2
23       ratingscount     S1       2
24       ratingscount     S2       2
25       ratingscount     S3       2
26        ratingvalue     S1       2
I seguenti LAT in SOURCE sono mappati in più GAT
                LAT SOURCE  counts
0            Autors     S2       2
3     NumberOfPages     S3       2
5   assessmentvalue     S3       3
10    nookbookprice     S1       2
12            pages     S2       2
13   paperbackprice     S1       2
14   paperbackprice     S2       2
15   paperbackprice     S3       2
16            price     S2       2

different tests

In [ ]:
ValutazioneMatchTable = pd.DataFrame(columns=['MT', 'TP', 'FP', 'FN', 'P', 'R', 'F'])
ValutazioneMatchTable

,MT,TP,FP,FN,P,R,F


In [ ]:
#96% precision 19 False Negatives
#Difeniamo la funzione per la matching Table
def CalcoloMatchingTable(TableL:pd.DataFrame,TableR:pd.DataFrame):
        SimTableA = levenshtein_label_based_similarity(TableL, TableR)
        #SimTableA = value_overlap_extended_jaccard_LEV(TableL, TableR,0.55)
        #SimTableB = jaro_label_based_similarity(TableL, TableR)
        SimTableB = babelnet_label_based_similarity(TableL,TableR)
        #SimTableB = value_overlap_sim(GlobalSchema, Sources[y])
        SimTableC=  value_overlap_simjoin_jaccard(TableL, TableR, 0.55)

        # combiner
        #SimTable = avg_sim_table([SimTableA,SimTableB,SimTableC])
        # SimTable = min_sim_table([SimTableA,SimTableB,SimTableC])
        #SimTable = max_sim_table([SimTableA,SimTableB,SimTableC])

        # Weighted-sum
        SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.2,0.6] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )

        # dalla tabella di similarità alle corrispondenze

        MatchTable= thresholding(SimTable, 0.55)

        #MatchTable = top_K(SimTable,2,'A')
        MatchTable = top_1(MatchTable,'A')

         # global mapping
        #MatchTable = stable_marriage(MatchTable)
        #MatchTable = simmetric_best_match(MatchTable)
        return MatchTable

In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)
ValutazioneMatchTable = pd.concat([ValutazioneMatchTable,Valuta(toA_B(GMT_GS),toA_B(GMT)).rename(index={0: 'lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6'})])
ValutazioneMatchTable

100%|██████████| 3/3 [00:05<00:00,  1.93s/it]


,MT,TP,FP,FN,P,R,F
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6,25,24,1,19,0.96,0.5581,0.7059


In [ ]:
#96% precision and 16FN
#Difeniamo la funzione per la matching Table
def CalcoloMatchingTable(TableL:pd.DataFrame,TableR:pd.DataFrame):
        SimTableA = levenshtein_label_based_similarity(TableL, TableR)
        #SimTableA = value_overlap_extended_jaccard_LEV(TableL, TableR,0.55)
        #SimTableB = jaro_label_based_similarity(TableL, TableR)
        SimTableB = babelnet_label_based_similarity(TableL,TableR)
        #SimTableB = value_overlap_sim(GlobalSchema, Sources[y])
        SimTableC=  value_overlap_simjoin_jaccard(TableL, TableR, 0.50)

        # combiner
        #SimTable = avg_sim_table([SimTableA,SimTableB,SimTableC])
        # SimTable = min_sim_table([SimTableA,SimTableB,SimTableC])
        #SimTable = max_sim_table([SimTableA,SimTableB,SimTableC])

        # Weighted-sum
        SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.2,0.6] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )

        # dalla tabella di similarità alle corrispondenze

        MatchTable= thresholding(SimTable, 0.5)

        #MatchTable = top_K(SimTable,2,'A')
        MatchTable = top_1(MatchTable,'A')

         # global mapping
        #MatchTable = stable_marriage(MatchTable)
        #MatchTable = simmetric_best_match(MatchTable)
        return MatchTable


In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)
ValutazioneMatchTable = pd.concat([ValutazioneMatchTable,Valuta(toA_B(GMT_GS),toA_B(GMT)).rename(index={0: 'lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6 treshold=0.5'})])
ValutazioneMatchTable

100%|██████████| 3/3 [00:05<00:00,  1.79s/it]


,MT,TP,FP,FN,P,R,F
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6,25,24,1,19,0.9600,0.5581,0.7059
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6 treshold=0.5,28,27,1,16,0.9643,0.6279,0.7606


In [ ]:
#f 0.5
def CalcoloMatchingTable(TableL:pd.DataFrame,TableR:pd.DataFrame):
        #SimTableA = levenshtein_label_based_similarity(TableL, TableR)
        SimTableA = value_overlap_extended_jaccard_LEV(TableL, TableR,0.3)
        #SimTableB = jaro_label_based_similarity(TableL, TableR)
        SimTableB = babelnet_label_based_similarity(TableL,TableR)
        #SimTableB = value_overlap_sim(GlobalSchema, Sources[y])
        SimTableC=  value_overlap_simjoin_jaccard(TableL, TableR, 0.30)

        # combiner
        #SimTable = avg_sim_table([SimTableA,SimTableB,SimTableC])
        # SimTable = min_sim_table([SimTableA,SimTableB,SimTableC])
        #SimTable = max_sim_table([SimTableA,SimTableB,SimTableC])

        # Weighted-sum
        SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.45,0.1,0.45] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )

        # dalla tabella di similarità alle corrispondenze

        MatchTable= thresholding(SimTable, 0.3)

        #MatchTable = top_K(SimTable,2,'A')
        MatchTable = top_1(MatchTable,'A')

         # global mapping
        #MatchTable = stable_marriage(MatchTable)
        #MatchTable = simmetric_best_match(MatchTable)
        return MatchTable

In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)
ValutazioneMatchTable = pd.concat([ValutazioneMatchTable,Valuta(toA_B(GMT_GS),toA_B(GMT)).rename(index={0: 'lev_overlap*0.45+babelnet*01+jaccard_overlap*0.45 treshold=0.3 fudge=0.5'})])
ValutazioneMatchTable

100%|██████████| 3/3 [00:14<00:00,  4.85s/it]


,MT,TP,FP,FN,P,R,F
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6,25,24,1,19,0.9600,0.5581,0.7059
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6 treshold=0.5,28,27,1,16,0.9643,0.6279,0.7606
lev_overlap*0.45+babelnet*01+jaccard_overlap*0.45 treshold=0.3 fudge=0.5,37,31,6,12,0.8378,0.7209,0.7750


In [ ]:
#label based matching
def CalcoloMatchingTable(TableL:pd.DataFrame,TableR:pd.DataFrame):
        SimTableA = levenshtein_label_based_similarity(TableL, TableR)
        #SimTableA = value_overlap_extended_jaccard_LEV(TableL, TableR,0.3)
        #SimTableB = jaro_label_based_similarity(TableL, TableR)
        SimTableB = babelnet_label_based_similarity(TableL,TableR)
        #SimTableB = value_overlap_sim(GlobalSchema, Sources[y])
        SimTableC=  jaccard_label_based_similarity(TableL, TableR)

        # combiner
        #SimTable = avg_sim_table([SimTableA,SimTableB,SimTableC])
        # SimTable = min_sim_table([SimTableA,SimTableB,SimTableC])
        #SimTable = max_sim_table([SimTableA,SimTableB,SimTableC])

        # Weighted-sum
        SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.25,0.5,0.25] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )

        # dalla tabella di similarità alle corrispondenze

        MatchTable= thresholding(SimTable, 0.5)

        #MatchTable = top_K(SimTable,2,'A')
        MatchTable = top_1(MatchTable,'A')

         # global mapping
        #MatchTable = stable_marriage(MatchTable)
        #MatchTable = simmetric_best_match(MatchTable)
        return MatchTable

In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)
ValutazioneMatchTable = pd.concat([ValutazioneMatchTable,Valuta(toA_B(GMT_GS),toA_B(GMT)).rename(index={0: 'lev_label*0.35+babelnet*0.5+jaccard_overlap*0.25 treshold=0.5 fudge=0.5'})])
ValutazioneMatchTable

100%|██████████| 3/3 [00:01<00:00,  2.58it/s]


,MT,TP,FP,FN,P,R,F
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6,25,24,1,19,0.9600,0.5581,0.7059
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6 treshold=0.5,28,27,1,16,0.9643,0.6279,0.7606
lev_overlap*0.45+babelnet*01+jaccard_overlap*0.45 treshold=0.3 fudge=0.5,37,31,6,12,0.8378,0.7209,0.7750
lev_label*0.35+babelnet*0.5+jaccard_overlap*0.25 treshold=0.5 fudge=0.5,31,25,6,18,0.8065,0.5814,0.6757


In [ ]:
#label based matching
def CalcoloMatchingTable(TableL:pd.DataFrame,TableR:pd.DataFrame):
        SimTableA = levenshtein_label_based_similarity(TableL, TableR)
        #SimTableA = value_overlap_extended_jaccard_LEV(TableL, TableR,0.3)
        #SimTableB = jaro_label_based_similarity(TableL, TableR)
        SimTableB = babelnet_label_based_similarity(TableL,TableR)
        #SimTableB = value_overlap_sim(GlobalSchema, Sources[y])
        SimTableC=  jaccard_label_based_similarity(TableL, TableR)

        # combiner
        #SimTable = avg_sim_table([SimTableA,SimTableB,SimTableC])
        # SimTable = min_sim_table([SimTableA,SimTableB,SimTableC])
        #SimTable = max_sim_table([SimTableA,SimTableB,SimTableC])

        # Weighted-sum
        SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.15,0.5,0.35] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )

        # dalla tabella di similarità alle corrispondenze

        MatchTable= thresholding(SimTable, 0.5)

        #MatchTable = top_K(SimTable,2,'A')
        MatchTable = top_1(MatchTable,'A')

         # global mapping
        #MatchTable = stable_marriage(MatchTable)
        #MatchTable = simmetric_best_match(MatchTable)
        return MatchTable

In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)
ValutazioneMatchTable = pd.concat([ValutazioneMatchTable,Valuta(toA_B(GMT_GS),toA_B(GMT)).rename(index={0: 'lev_label*0.15+babelnet*0.5+jaccard_overlap*0.35 treshold=0.5 fudge=0.5'})])
ValutazioneMatchTable

100%|██████████| 3/3 [00:00<00:00,  5.24it/s]


,MT,TP,FP,FN,P,R,F
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6,25,24,1,19,0.9600,0.5581,0.7059
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6 treshold=0.5,28,27,1,16,0.9643,0.6279,0.7606
lev_overlap*0.45+babelnet*01+jaccard_overlap*0.45 treshold=0.3 fudge=0.5,37,31,6,12,0.8378,0.7209,0.7750
lev_label*0.35+babelnet*0.5+jaccard_overlap*0.25 treshold=0.5 fudge=0.5,31,25,6,18,0.8065,0.5814,0.6757
lev_label*0.15+babelnet*0.5+jaccard_overlap*0.35 treshold=0.5 fudge=0.5,29,24,5,19,0.8276,0.5581,0.6667


In [ ]:
#label based matching
def CalcoloMatchingTable(TableL:pd.DataFrame,TableR:pd.DataFrame):
        SimTableA = levenshtein_label_based_similarity(TableL, TableR)
        #SimTableA = value_overlap_extended_jaccard_LEV(TableL, TableR,0.3)
        #SimTableB = jaro_label_based_similarity(TableL, TableR)
        SimTableB = babelnet_label_based_similarity(TableL,TableR)
        #SimTableB = value_overlap_sim(GlobalSchema, Sources[y])
        SimTableC=  jaccard_label_based_similarity(TableL, TableR)

        # combiner
        #SimTable = avg_sim_table([SimTableA,SimTableB,SimTableC])
        # SimTable = min_sim_table([SimTableA,SimTableB,SimTableC])
        #SimTable = max_sim_table([SimTableA,SimTableB,SimTableC])

        # Weighted-sum
        SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.15,0.5,0.35] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )

        # dalla tabella di similarità alle corrispondenze

        MatchTable= thresholding(SimTable, 0.45)

        #MatchTable = top_K(SimTable,2,'A')
        MatchTable = top_1(MatchTable,'A')

         # global mapping
        #MatchTable = stable_marriage(MatchTable)
        #MatchTable = simmetric_best_match(MatchTable)
        return MatchTable

In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)
ValutazioneMatchTable = pd.concat([ValutazioneMatchTable,Valuta(toA_B(GMT_GS),toA_B(GMT)).rename(index={0: 'lev_label*0.15+babelnet*0.5+jaccard_overlap*0.35 treshold=0.45 fudge=0.5'})])
ValutazioneMatchTable

100%|██████████| 3/3 [00:00<00:00,  4.73it/s]


,MT,TP,FP,FN,P,R,F
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6,25,24,1,19,0.9600,0.5581,0.7059
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6 treshold=0.5,28,27,1,16,0.9643,0.6279,0.7606
lev_overlap*0.45+babelnet*01+jaccard_overlap*0.45 treshold=0.3 fudge=0.5,37,31,6,12,0.8378,0.7209,0.7750
lev_label*0.35+babelnet*0.5+jaccard_overlap*0.25 treshold=0.5 fudge=0.5,31,25,6,18,0.8065,0.5814,0.6757
lev_label*0.15+babelnet*0.5+jaccard_overlap*0.35 treshold=0.5 fudge=0.5,29,24,5,19,0.8276,0.5581,0.6667
lev_label*0.15+babelnet*0.5+jaccard_overlap*0.35 treshold=0.45 fudge=0.5,32,26,6,17,0.8125,0.6047,0.6933


In [ ]:
#label based matching
def CalcoloMatchingTable(TableL:pd.DataFrame,TableR:pd.DataFrame):
        SimTableA = levenshtein_label_based_similarity(TableL, TableR)
        #SimTableA = value_overlap_extended_jaccard_LEV(TableL, TableR,0.3)
        #SimTableB = jaro_label_based_similarity(TableL, TableR)
        SimTableB = babelnet_label_based_similarity(TableL,TableR)
        #SimTableB = value_overlap_sim(GlobalSchema, Sources[y])
        SimTableC=  jaccard_label_based_similarity(TableL, TableR)

        # combiner
        #SimTable = avg_sim_table([SimTableA,SimTableB,SimTableC])
        # SimTable = min_sim_table([SimTableA,SimTableB,SimTableC])
        #SimTable = max_sim_table([SimTableA,SimTableB,SimTableC])

        # Weighted-sum
        SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.55,0.25] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )

        # dalla tabella di similarità alle corrispondenze

        MatchTable= thresholding(SimTable, 0.4)

        #MatchTable = top_K(SimTable,2,'A')
        MatchTable = top_1(MatchTable,'A')

         # global mapping
        #MatchTable = stable_marriage(MatchTable)
        #MatchTable = simmetric_best_match(MatchTable)
        return MatchTable

In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)
ValutazioneMatchTable = pd.concat([ValutazioneMatchTable,Valuta(toA_B(GMT_GS),toA_B(GMT)).rename(index={0: 'lev_label*0.2+babelnet*0.55+jaccard_overlap*0.25 treshold=0.4 fudge=0.5'})])
ValutazioneMatchTable

100%|██████████| 3/3 [00:01<00:00,  2.22it/s]


,MT,TP,FP,FN,P,R,F
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6,25,24,1,19,0.9600,0.5581,0.7059
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6 treshold=0.5,28,27,1,16,0.9643,0.6279,0.7606
lev_overlap*0.45+babelnet*01+jaccard_overlap*0.45 treshold=0.3 fudge=0.5,37,31,6,12,0.8378,0.7209,0.7750
lev_label*0.35+babelnet*0.5+jaccard_overlap*0.25 treshold=0.5 fudge=0.5,31,25,6,18,0.8065,0.5814,0.6757
lev_label*0.15+babelnet*0.5+jaccard_overlap*0.35 treshold=0.5 fudge=0.5,29,24,5,19,0.8276,0.5581,0.6667
lev_label*0.15+babelnet*0.5+jaccard_overlap*0.35 treshold=0.45 fudge=0.5,32,26,6,17,0.8125,0.6047,0.6933
lev_label*0.2+babelnet*0.55+jaccard_overlap*0.25 treshold=0.4 fudge=0.5,34,27,7,16,0.7941,0.6279,0.7013


In [ ]:
#f 0.5
def CalcoloMatchingTable(TableL:pd.DataFrame,TableR:pd.DataFrame):
        #SimTableA = levenshtein_label_based_similarity(TableL, TableR)
        SimTableA = value_overlap_extended_jaccard_LEV(TableL, TableR,0.3)
        SimTableB = jaro_label_based_similarity(TableL, TableR)
        #SimTableB = babelnet_label_based_similarity(TableL,TableR)
        #SimTableB = value_overlap_sim(GlobalSchema, Sources[y])
        SimTableC=  value_overlap_simjoin_jaccard(TableL, TableR, 0.30)

        # combiner
        #SimTable = avg_sim_table([SimTableA,SimTableB,SimTableC])
        # SimTable = min_sim_table([SimTableA,SimTableB,SimTableC])
        #SimTable = max_sim_table([SimTableA,SimTableB,SimTableC])

        # Weighted-sum
        SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.45,0.1,0.45] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )

        # dalla tabella di similarità alle corrispondenze

        MatchTable= thresholding(SimTable, 0.3)

        #MatchTable = top_K(SimTable,2,'A')
        MatchTable = top_1(MatchTable,'A')

         # global mapping
        #MatchTable = stable_marriage(MatchTable)
        #MatchTable = simmetric_best_match(MatchTable)
        return MatchTable

In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)
ValutazioneMatchTable = pd.concat([ValutazioneMatchTable,Valuta(toA_B(GMT_GS),toA_B(GMT)).rename(index={0: 'jac_lev_overlap*0.45+jaro_label*0.1+jaccard_overlap*0.45 treshold=0.3 fudge=0.5'})])
ValutazioneMatchTable

100%|██████████| 3/3 [00:13<00:00,  4.40s/it]


,MT,TP,FP,FN,P,R,F
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6,25,24,1,19,0.9600,0.5581,0.7059
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6 treshold=0.5,28,27,1,16,0.9643,0.6279,0.7606
lev_overlap*0.45+babelnet*01+jaccard_overlap*0.45 treshold=0.3 fudge=0.5,37,31,6,12,0.8378,0.7209,0.7750
lev_label*0.35+babelnet*0.5+jaccard_overlap*0.25 treshold=0.5 fudge=0.5,31,25,6,18,0.8065,0.5814,0.6757
lev_label*0.15+babelnet*0.5+jaccard_overlap*0.35 treshold=0.5 fudge=0.5,29,24,5,19,0.8276,0.5581,0.6667
lev_label*0.15+babelnet*0.5+jaccard_overlap*0.35 treshold=0.45 fudge=0.5,32,26,6,17,0.8125,0.6047,0.6933
lev_label*0.2+babelnet*0.55+jaccard_overlap*0.25 treshold=0.4 fudge=0.5,34,27,7,16,0.7941,0.6279,0.7013
jac_lev_overlap*0.45+jaro_label*0.1+jaccard_overlap*0.45 treshold=0.3 fudge=0.5,37,31,6,12,0.8378,0.7209,0.7750


In [ ]:
def CalcoloMatchingTable(TableL:pd.DataFrame,TableR:pd.DataFrame):
        #SimTableA = levenshtein_label_based_similarity(TableL, TableR)
        SimTableA = value_overlap_extended_jaccard_LEV(TableL, TableR,0.4)
        SimTableB = jaro_label_based_similarity(TableL, TableR)
        #SimTableB = babelnet_label_based_similarity(TableL,TableR)
        #SimTableB = value_overlap_sim(GlobalSchema, Sources[y])
        SimTableC=  value_overlap_simjoin_jaccard(TableL, TableR, 0.4)

        # combiner
        SimTable = avg_sim_table([SimTableA,SimTableB,SimTableC])
        # SimTable = min_sim_table([SimTableA,SimTableB,SimTableC])
        #SimTable = max_sim_table([SimTableA,SimTableB,SimTableC])

        # Weighted-sum
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.45,0.1,0.45] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )

        # dalla tabella di similarità alle corrispondenze

        MatchTable= thresholding(SimTable, 0.4)

        #MatchTable = top_K(SimTable,2,'A')
        MatchTable = top_1(MatchTable,'A')

         # global mapping
        #MatchTable = stable_marriage(MatchTable)
        #MatchTable = simmetric_best_match(MatchTable)
        return MatchTable

In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)
ValutazioneMatchTable = pd.concat([ValutazioneMatchTable,Valuta(toA_B(GMT_GS),toA_B(GMT)).rename(index={0: 'jac_lev_overlap_jaro_label_jaccard_overlap_avg treshold=0.4 fudge=0.5'})])
ValutazioneMatchTable

100%|██████████| 3/3 [00:12<00:00,  4.33s/it]


,MT,TP,FP,FN,P,R,F
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6,25,24,1,19,0.9600,0.5581,0.7059
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6 treshold=0.5,28,27,1,16,0.9643,0.6279,0.7606
lev_overlap*0.45+babelnet*01+jaccard_overlap*0.45 treshold=0.3 fudge=0.5,37,31,6,12,0.8378,0.7209,0.7750
lev_label*0.35+babelnet*0.5+jaccard_overlap*0.25 treshold=0.5 fudge=0.5,31,25,6,18,0.8065,0.5814,0.6757
lev_label*0.15+babelnet*0.5+jaccard_overlap*0.35 treshold=0.5 fudge=0.5,29,24,5,19,0.8276,0.5581,0.6667
lev_label*0.15+babelnet*0.5+jaccard_overlap*0.35 treshold=0.45 fudge=0.5,32,26,6,17,0.8125,0.6047,0.6933
lev_label*0.2+babelnet*0.55+jaccard_overlap*0.25 treshold=0.4 fudge=0.5,34,27,7,16,0.7941,0.6279,0.7013
jac_lev_overlap*0.45+jaro_label*0.1+jaccard_overlap*0.45 treshold=0.3 fudge=0.5,37,31,6,12,0.8378,0.7209,0.7750
jac_lev_overlap_jaro_label_jaccard_overlap_avg treshold=0.4 fudge=0.5,36,31,5,12,0.8611,0.7209,0.7848


In [ ]:
def CalcoloMatchingTable(TableL:pd.DataFrame,TableR:pd.DataFrame):
        #SimTableA = levenshtein_label_based_similarity(TableL, TableR)
        SimTableA = value_overlap_extended_jaccard_LEV(TableL, TableR,0.5)
        SimTableB = jaro_label_based_similarity(TableL, TableR)
        #SimTableB = babelnet_label_based_similarity(TableL,TableR)
        #SimTableB = value_overlap_sim(GlobalSchema, Sources[y])
        SimTableC=  value_overlap_simjoin_jaccard(TableL, TableR, 0.5)

        # combiner
        #SimTable = avg_sim_table([SimTableA,SimTableB,SimTableC])
        # SimTable = min_sim_table([SimTableA,SimTableB,SimTableC])
        SimTable = max_sim_table([SimTableA,SimTableB,SimTableC])

        # Weighted-sum
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.45,0.1,0.45] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )

        # dalla tabella di similarità alle corrispondenze

        MatchTable= thresholding(SimTable, 0.55)

        #MatchTable = top_K(SimTable,2,'A')
        MatchTable = top_1(MatchTable,'A')

         # global mapping
        #MatchTable = stable_marriage(MatchTable)
        #MatchTable = simmetric_best_match(MatchTable)
        return MatchTable

In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)
ValutazioneMatchTable = pd.concat([ValutazioneMatchTable,Valuta(toA_B(GMT_GS),toA_B(GMT)).rename(index={0: 'jac_lev_overlap_jaro_label_jaccard_overlap_max treshold=0.55'})])
ValutazioneMatchTable

100%|██████████| 3/3 [00:12<00:00,  4.31s/it]


,MT,TP,FP,FN,P,R,F
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6,25,24,1,19,0.9600,0.5581,0.7059
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6 treshold=0.5,28,27,1,16,0.9643,0.6279,0.7606
lev_overlap*0.45+babelnet*01+jaccard_overlap*0.45 treshold=0.3 fudge=0.5,37,31,6,12,0.8378,0.7209,0.7750
lev_label*0.35+babelnet*0.5+jaccard_overlap*0.25 treshold=0.5 fudge=0.5,31,25,6,18,0.8065,0.5814,0.6757
lev_label*0.15+babelnet*0.5+jaccard_overlap*0.35 treshold=0.5 fudge=0.5,29,24,5,19,0.8276,0.5581,0.6667
lev_label*0.15+babelnet*0.5+jaccard_overlap*0.35 treshold=0.45 fudge=0.5,32,26,6,17,0.8125,0.6047,0.6933
lev_label*0.2+babelnet*0.55+jaccard_overlap*0.25 treshold=0.4 fudge=0.5,34,27,7,16,0.7941,0.6279,0.7013
jac_lev_overlap*0.45+jaro_label*0.1+jaccard_overlap*0.45 treshold=0.3 fudge=0.5,37,31,6,12,0.8378,0.7209,0.7750
jac_lev_overlap_jaro_label_jaccard_overlap_avg treshold=0.4 fudge=0.5,36,31,5,12,0.8611,0.7209,0.7848
jac_lev_overlap_jaro_label_jaccard_overlap_max treshold=0.55,39,29,10,14,0.7436,0.6744,0.7073


In [ ]:
#label based matching
def CalcoloMatchingTable(TableL:pd.DataFrame,TableR:pd.DataFrame):
        SimTableA = levenshtein_label_based_similarity(TableL, TableR)
        #SimTableA = value_overlap_extended_jaccard_LEV(TableL, TableR,0.3)
        #SimTableB = jaro_label_based_similarity(TableL, TableR)
        SimTableB = babelnet_label_based_similarity(TableL,TableR)
        #SimTableB = value_overlap_sim(GlobalSchema, Sources[y])
        SimTableC=  jaccard_label_based_similarity(TableL, TableR)

        # combiner
        #SimTable = avg_sim_table([SimTableA,SimTableB,SimTableC])
        # SimTable = min_sim_table([SimTableA,SimTableB,SimTableC])
        SimTable = max_sim_table([SimTableA,SimTableB,SimTableC])

        # Weighted-sum
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.25,0.5,0.25] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )

        # dalla tabella di similarità alle corrispondenze

        MatchTable= thresholding(SimTable, 0.3)

        #MatchTable = top_K(SimTable,2,'A')
        MatchTable = top_1(MatchTable,'A')

         # global mapping
        #MatchTable = stable_marriage(MatchTable)
        #MatchTable = simmetric_best_match(MatchTable)
        return MatchTable

In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)
ValutazioneMatchTable = pd.concat([ValutazioneMatchTable,Valuta(toA_B(GMT_GS),toA_B(GMT)).rename(index={0: 'lev_label_babelnet_label_jaccard_overlap_max treshold=0.3'})])
ValutazioneMatchTable

100%|██████████| 3/3 [00:00<00:00,  3.15it/s]


,MT,TP,FP,FN,P,R,F
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6,25,24,1,19,0.9600,0.5581,0.7059
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6 treshold=0.5,28,27,1,16,0.9643,0.6279,0.7606
lev_overlap*0.45+babelnet*01+jaccard_overlap*0.45 treshold=0.3 fudge=0.5,37,31,6,12,0.8378,0.7209,0.7750
lev_label*0.35+babelnet*0.5+jaccard_overlap*0.25 treshold=0.5 fudge=0.5,31,25,6,18,0.8065,0.5814,0.6757
lev_label*0.15+babelnet*0.5+jaccard_overlap*0.35 treshold=0.5 fudge=0.5,29,24,5,19,0.8276,0.5581,0.6667
lev_label*0.15+babelnet*0.5+jaccard_overlap*0.35 treshold=0.45 fudge=0.5,32,26,6,17,0.8125,0.6047,0.6933
lev_label*0.2+babelnet*0.55+jaccard_overlap*0.25 treshold=0.4 fudge=0.5,34,27,7,16,0.7941,0.6279,0.7013
jac_lev_overlap*0.45+jaro_label*0.1+jaccard_overlap*0.45 treshold=0.3 fudge=0.5,37,31,6,12,0.8378,0.7209,0.7750
jac_lev_overlap_jaro_label_jaccard_overlap_avg treshold=0.4 fudge=0.5,36,31,5,12,0.8611,0.7209,0.7848
jac_lev_overlap_jaro_label_jaccard_overlap_max treshold=0.55,39,29,10,14,0.7436,0.6744,0.7073


In [ ]:
#f
def CalcoloMatchingTable(TableL:pd.DataFrame,TableR:pd.DataFrame):
        #SimTableA = levenshtein_label_based_similarity(TableL, TableR)
        SimTableA = value_overlap_extended_jaccard_LEV(TableL, TableR,0.3)
        #SimTableB = jaro_label_based_similarity(TableL, TableR)
        SimTableB = babelnet_label_based_similarity(TableL,TableR)
        #SimTableB = value_overlap_sim(GlobalSchema, Sources[y])
        SimTableC=  value_overlap_simjoin_jaccard(TableL, TableR, 0.30)

        # combiner
        SimTable = avg_sim_table([SimTableA,SimTableB,SimTableC])
        # SimTable = min_sim_table([SimTableA,SimTableB,SimTableC])
        #SimTable = max_sim_table([SimTableA,SimTableB,SimTableC])

        # Weighted-sum
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.45,0.1,0.45] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )
        #SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )

        # dalla tabella di similarità alle corrispondenze

        MatchTable= thresholding(SimTable, 0.4)

        #MatchTable = top_K(SimTable,2,'A')
        MatchTable = top_1(MatchTable,'A')

         # global mapping
        #MatchTable = stable_marriage(MatchTable)
        #MatchTable = simmetric_best_match(MatchTable)
        return MatchTable

In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)
ValutazioneMatchTable = pd.concat([ValutazioneMatchTable,Valuta(toA_B(GMT_GS),toA_B(GMT)).rename(index={0: 'ex_jaccard_overlap_babelnet_label_jaccard_overlap_avg treshold=0.4'})])
ValutazioneMatchTable

100%|██████████| 3/3 [00:14<00:00,  4.81s/it]


,MT,TP,FP,FN,P,R,F
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6,25,24,1,19,0.9600,0.5581,0.7059
lev_overlap*0.2+babelnet*02+jaccard_overlap*0.6 treshold=0.5,28,27,1,16,0.9643,0.6279,0.7606
lev_overlap*0.45+babelnet*01+jaccard_overlap*0.45 treshold=0.3 fudge=0.5,37,31,6,12,0.8378,0.7209,0.7750
lev_label*0.35+babelnet*0.5+jaccard_overlap*0.25 treshold=0.5 fudge=0.5,31,25,6,18,0.8065,0.5814,0.6757
lev_label*0.15+babelnet*0.5+jaccard_overlap*0.35 treshold=0.5 fudge=0.5,29,24,5,19,0.8276,0.5581,0.6667
lev_label*0.15+babelnet*0.5+jaccard_overlap*0.35 treshold=0.45 fudge=0.5,32,26,6,17,0.8125,0.6047,0.6933
lev_label*0.2+babelnet*0.55+jaccard_overlap*0.25 treshold=0.4 fudge=0.5,34,27,7,16,0.7941,0.6279,0.7013
jac_lev_overlap*0.45+jaro_label*0.1+jaccard_overlap*0.45 treshold=0.3 fudge=0.5,37,31,6,12,0.8378,0.7209,0.7750
jac_lev_overlap_jaro_label_jaccard_overlap_avg treshold=0.4 fudge=0.5,36,31,5,12,0.8611,0.7209,0.7848
jac_lev_overlap_jaro_label_jaccard_overlap_max treshold=0.55,39,29,10,14,0.7436,0.6744,0.7073


In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)

In [ ]:
ValutazioneMatchTable = pd.DataFrame(columns=['MT', 'TP', 'FP', 'FN', 'P', 'R', 'F'])
ValutazioneMatchTable

,MT,TP,FP,FN,P,R,F


In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)
ValutazioneMatchTable = pd.concat([ValutazioneMatchTable,Valuta(toA_B(GMT_GS),toA_B(GMT)).rename(index={0: 'Seconda Prova'})], ignore_index=True)
#.append(


#   )
#       MATCH = pd.concat([MATCH, x])
#SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.4,0.3,0.3] )
#TOP 2
ValutazioneMatchTable

In [ ]:
#.append(


#   )
#       MATCH = pd.concat([MATCH, x])
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)
ValutazioneMatchTable = pd.concat([ValutazioneMatchTable,Valuta(toA_B(GMT_GS),toA_B(GMT)).rename(index={0: 'Seconda Prova'})], ignore_index=True)
#.append(


#   )
#       MATCH = pd.concat([MATCH, x])
#SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.4,0.3,0.3] )
#TOP 2
#Tresholding raised from 0.4 to 0.55
ValutazioneMatchTable

  0%|          | 0/3 [00:00<?, ?it/s]

il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!


 33%|███▎      | 1/3 [00:05<00:11,  5.82s/it]

il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!


 67%|██████▋   | 2/3 [00:10<00:05,  5.01s/it]

il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!


100%|██████████| 3/3 [00:13<00:00,  4.66s/it]


,MT,TP,FP,FN,P,R,F
0,33,27,6,16,0.8182,0.6279,0.7105
1,78,38,40,5,0.4872,0.8837,0.6281
2,78,38,40,5,0.4872,0.8837,0.6281
3,78,38,40,5,0.4872,0.8837,0.6281
4,78,37,41,6,0.4744,0.8605,0.6116
...,...,...,...,...,...,...,...
78,37,31,6,12,0.8378,0.7209,0.7750
79,37,31,6,12,0.8378,0.7209,0.7750
80,37,30,7,13,0.8108,0.6977,0.7500
81,37,31,6,12,0.8378,0.7209,0.7750


In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)
ValutazioneMatchTable = pd.concat([ValutazioneMatchTable,Valuta(toA_B(GMT_GS),toA_B(GMT)).rename(index={0: 'Seconda Prova'})], ignore_index=True)
#.append(


#   )
#       MATCH = pd.concat([MATCH, x])
#SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.2,0.5,0.3] )
#TOP 2
#Tresholding raised from 0.4 to 0.55
ValutazioneMatchTable

  0%|          | 0/3 [00:00<?, ?it/s]

il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!


 33%|███▎      | 1/3 [00:02<00:05,  2.80s/it]

il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!


 67%|██████▋   | 2/3 [00:04<00:02,  2.17s/it]

il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!


100%|██████████| 3/3 [00:06<00:00,  2.04s/it]


,MT,TP,FP,FN,P,R,F
0,33,27,6,16,0.8182,0.6279,0.7105
1,78,38,40,5,0.4872,0.8837,0.6281
2,78,38,40,5,0.4872,0.8837,0.6281
3,78,38,40,5,0.4872,0.8837,0.6281
4,78,37,41,6,0.4744,0.8605,0.6116
5,78,38,40,5,0.4872,0.8837,0.6281
6,78,38,40,5,0.4872,0.8837,0.6281
7,78,38,40,5,0.4872,0.8837,0.6281
8,78,40,38,3,0.5128,0.9302,0.6612
9,78,39,39,4,0.5000,0.9070,0.6446


In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)
ValutazioneMatchTable = pd.concat([ValutazioneMatchTable,Valuta(toA_B(GMT_GS),toA_B(GMT)).rename(index={0: 'Seconda Prova'})], ignore_index=True)
#SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.1,0.5,0.4] )

#TOP 2
#Tresholding raised from 0.4 to 0.55
ValutazioneMatchTable




  0%|          | 0/3 [00:00<?, ?it/s]

il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!


 33%|███▎      | 1/3 [00:02<00:04,  2.20s/it]

il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!


 67%|██████▋   | 2/3 [00:04<00:02,  2.19s/it]

il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!


100%|██████████| 3/3 [00:06<00:00,  2.04s/it]


,MT,TP,FP,FN,P,R,F
0,33,27,6,16,0.8182,0.6279,0.7105
1,78,38,40,5,0.4872,0.8837,0.6281
2,78,38,40,5,0.4872,0.8837,0.6281
3,78,38,40,5,0.4872,0.8837,0.6281
4,78,37,41,6,0.4744,0.8605,0.6116
5,78,38,40,5,0.4872,0.8837,0.6281


In [ ]:
GMT=CalcoloGlobalMatchingTable(SOURCES,GlobalSchema)
ValutazioneMatchTable = pd.concat([ValutazioneMatchTable,Valuta(toA_B(GMT_GS),toA_B(GMT)).rename(index={0: 'Seconda Prova'})], ignore_index=True)
#SimTable = Weighted_sum([SimTableA,SimTableB,SimTableC], [0.1,0.5,0.4] )

#TOP 2
#        MatchTable= thresholding(SimTable, 0.6)
#        SimTableC=  value_overlap_simjoin_jaccard(TableL, TableR, 0.7)

ValutazioneMatchTable

  0%|          | 0/3 [00:00<?, ?it/s]

il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!
il nodo ratingscount non c'è!


 33%|███▎      | 1/3 [00:02<00:04,  2.32s/it]

il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!
il nodo Autors non c'è!
il nodo YearEditor non c'è!


 67%|██████▋   | 2/3 [00:04<00:01,  1.96s/it]

il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!
il nodo PubblicationInfo non c'è!


100%|██████████| 3/3 [00:05<00:00,  1.82s/it]


,MT,TP,FP,FN,P,R,F
0,33,27,6,16,0.8182,0.6279,0.7105
1,78,38,40,5,0.4872,0.8837,0.6281
2,78,38,40,5,0.4872,0.8837,0.6281
3,78,38,40,5,0.4872,0.8837,0.6281
4,78,37,41,6,0.4744,0.8605,0.6116
5,78,38,40,5,0.4872,0.8837,0.6281
6,78,38,40,5,0.4872,0.8837,0.6281
7,78,38,40,5,0.4872,0.8837,0.6281
8,78,40,38,3,0.5128,0.9302,0.6612
9,78,39,39,4,0.5000,0.9070,0.6446


In [ ]:
Vedi_Valuta(toA_B(GMT_GS),toA_B(GMT),'FN')

,A,B,_merge
3,nookbookprice,S1_paperbackprice,left_only
5,pages,S1_ratingscount,left_only
7,paperbackprice,S1_nookbookprice,left_only
9,productdimensions,S1_size,left_only
13,ratingscount,S1_ratingvalue,left_only
15,ratingvalue,S1_ratingscount,left_only
21,nookbookprice,S2_paperbackprice,left_only
23,pages,S2_valuation,left_only
25,paperbackprice,S2_price,left_only
28,ratingscount,S2_valuation,left_only
